In [1]:
import pandas as pd
import numpy as np

In [ ]:
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
benign_data = pd.read_csv(r'.\Data\CSV_benign.csv')
malware_data = pd.read_csv(r'.\Data\CSV_malware.csv', dtype = str, keep_default_na=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\3512400631.py:1: DtypeWarning: Columns (9,10,12,13,17,18,20,21,24,25,27,28,36) have mixed types. Specify dtype option on import or set low_memory=False.
  benign_data = pd.read_csv(r'.\Data\CSV_benign.csv')


In [4]:
len(benign_data.columns), len(malware_data.columns)

(38, 38)

In [5]:
for colname in (malware_data.columns.values):
    if colname in benign_data.columns.values:
        continue
    else: 
        print("Brak kolumny ", colname)

In [6]:
for colname in (benign_data.columns.values):
    if colname in malware_data.columns.values:
        continue
    else: 
        print("Brak kolumny ", colname)

## Merge df

In [7]:
benign_data['malware'] = 0
malware_data['malware'] = 1

In [8]:
data = pd.concat([benign_data, malware_data], axis=0, ignore_index=True)

In [9]:
len(data), len(malware_data) + len(benign_data)

(499134, 499134)

In [10]:
data.head(3)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware
0,US,15169.0,46,172.217.1.174,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0
1,NaN,NaN,299,NaN,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0
2,NaN,NaN,38,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0,0


### Data Preparation

Poprawa odnalezionych błędów w tabeli. Znalezione rekordy, których wartości nie były dopasowane do opisów w kolumnach skorygowano.

#### 'Country'

In [11]:
country_code = list(data['Country'].unique())

In [12]:
non_country_code = []

for code in country_code:
    if pd.isna(code):
        continue
    elif str(code).isdigit():
        non_country_code.append(code)

In [13]:
wrong_country_code = data[data['Country'].isin(non_country_code)]

In [14]:
wrong_country_code_malware = wrong_country_code[wrong_country_code['malware']==1]

In [15]:
wrong_country_code_malware.rename(columns={'Country': 'ASN', 
                      'ASN' : 'TTL' ,
                      'TTL' : 'Domain', 
                      'IP' : 'Country', 
                      'Domain' : 'IP'}, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\3487219340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_country_code_malware.rename(columns={'Country': 'ASN',


In [16]:
data[data['Country'].isin(non_country_code) & data['malware']==1] = wrong_country_code_malware

In [17]:
data.iloc[1990]

Country                                                              US
ASN                                                             14618.0
TTL                                                                 299
IP                                                         52.206.136.3
Domain                                                 b'archives.gov.'
State                                                               NaN
Registrant_Name                                                     NaN
Country.1                                                           NaN
Creation_Date_Time                                                  NaN
hex_32                                                                0
hex_8                                                                 0
Domain_Name                                                ARCHIVES.GOV
Alexa_Rank                                                       9662.0
subdomain                                                       

In [18]:
country_code = list(data['Country'].unique())
non_country_code = []

for code in country_code:
    if pd.isna(code):
        continue
    elif str(code).isdigit():
        non_country_code.append(code)

In [19]:
data['CountryCODE'] = data['Country']

In [20]:
pattern = {
 'aust' : 'AU',
 'australia' : 'AU',
 'austria' : 'AT',
 'argentina' : 'AR',
 'bangladesh' : 'BD',
 'belarus' : 'BY',
 'canada' : 'CA',
 'china' : 'CN',
 'cyprus' : 'CY',
 'czech republic' : 'CZ',
 'dominican republic' : 'DO',
 'germany' : 'DE',
 'hong kong' : 'HK',
 'italy' : 'IT',
 'india' : 'IN',
 'malaysia' : 'MY',
 'nepal' : 'NP',
 'nigeria' : 'NG',
 'spain' : 'ES',
 'seychelles' : 'SC',
 'switzerland' : 'CH',
 'turkey' : 'TR',
 'ukraine' : 'UA',
 'united states' : 'US',
 'usa' : 'US',
 'uganda' : 'UG',
 'france' : 'FR'}

In [21]:
for idx, row in data[data['CountryCODE'].isna() | data['CountryCODE'].isin(non_country_code)].iterrows():
    if pd.isna(row['Country.1']):
        continue
    elif row['Country.1'].isalpha and len(row['Country.1']) == 2:
        data.loc[idx, 'CountryCODE'] = row['Country.1']
    elif row['Country.1'].replace('.', '').lower() in pattern.keys():
        data.loc[idx, 'CountryCODE'] = pattern[row['Country.1'].replace('.', '').lower()]
    elif row['Country.1'].split(',')[0].replace('[', '').replace("'", '').replace("]", '').isalpha and len(row['Country.1'].split(',')[0].replace('[', '').replace("'", '').replace("]", '')) == 2:
        data.loc[idx, 'CountryCODE'] = row['Country.1'].split(',')[0].replace('[', '').replace("'", '').replace("]", '').upper()
    elif row['Country.1'].split(',')[0].replace('[', '').replace("'", '').replace("]", '') in pattern.keys():
        data.loc[idx, 'CountryCODE'] = pattern[row['Country.1'].split(',')[0].replace('[', '').replace("'", '').replace("]", '')]

In [22]:
data[data['CountryCODE'].isna() | data['CountryCODE'].isin(non_country_code)]

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
2,NaN,NaN,38,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0,0,NaN
11,NaN,NaN,95,NaN,b'youtube.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,NaN,8,youtube,0.0,-1.0,"['y', 'o', 'u', 't', 'u', 'b', 'e']",0.0,2.375,NaN,com,0,0,NaN,0.0,0,"[('youtube.com', 100), ('qq.com', 86)]",0,"['you', 'out', 'utu', 'tub', 'ube']","defaultdict(<class 'int'>, {'t': 1, 'u': 2, 'o...","['yo', 'ou', 'ut', 'tu', 'ub', 'be']",NaN,youtube,NaN,NaN,0,NaN
12,NaN,NaN,95,NaN,b'youtube.com.',NaN,NaN,NaN,2005-02-15 05:13:12,0,0,YOUTUBE.COM,2.0,0,NaN,8,youtube,0.0,-1.0,"['y', 'o', 'u', 't', 'u', 'b', 'e']",0.0,2.375,5546 days 10:40:46.327695,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('youtube.com', 100), ('qq.com', 86)]",0,"['you', 'out', 'utu', 'tub', 'ube']","defaultdict(<class 'int'>, {'t': 1, 'u': 2, 'o...","['yo', 'ou', 'ut', 'tu', 'ub', 'be']",MarkMonitor Inc.,youtube,4.0,-1.0,0,NaN
15,NaN,NaN,21599,NaN,b'www.youtube.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,11,youtube,0.0,-1.0,"['y', 'o', 'u', 't', 'u', 'b', 'e']",0.0,2.5308571531099555,NaN,com,0,0,NaN,0.0,0,"[('youtube.com', 100), ('qq.com', 86)]",0,"['you', 'out', 'utu', 'tub', 'ube']","defaultdict(<class 'int'>, {'t': 1, 'u': 2, 'o...","['yo', 'ou', 'ut', 'tu', 'ub', 'be']",NaN,youtube,NaN,NaN,0,NaN
20,NaN,NaN,299,NaN,b'www.linkedin.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,12,linked,0.0,-1.0,"['l', 'i', 'n', 'k', 'e', 'd', 'i', 'n']",0.0,2.556641667147437,NaN,com,0,0,NaN,0.0,0,"[('linkedin.com', 100), ('qq.com', 86)]",0,"['lin', 'ink', 'nke', 'ked', 'edi', 'din']","defaultdict(<class 'int'>, {'e': 1, 'i': 2, 'k...","['li', 'in', 'nk', 'ke', 'ed', 'di', 'in']",NaN,linkedin,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498935,NaN,NaN,3598,NaN,b'ww38.rail.us.',NaN,"['michael marcovici', 'REDACTED FOR PRIVACY']",NaN,2004-06-06 11:53:33,0,0,rail.us,-1.0,1,NaN,9,rail,0,-1.0,"['r', 'a', 'i', 'l']",0,2.5954888901709436,5797 days 15:45:30.203458,us,0,0,NaN,14.285714285714285,0,"[('mail.ru', 71), ('dailymail.co.uk', 61)]",0,"['rai', 'ail']","defaultdict(<class 'int'>, {'3': 1, 'i': 1, 'r...","['ra', 'ai', 'il']",NaN,rail,4.0,-1.0,1,NaN
498961,NaN,NaN,1399,NaN,b'www.costlessautoparts.com.au.',NaN,NaN,NaN,NaN,0,0,NaN,-1.0,1,NaN,21,parts,0,-1.0,"['c', 'o', 's', 't', 'l', 'e', 's', 's', 'a', ...",0,3.1588844500594413,0,com.au,0,0,NaN,0.0,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['cos', 'ost', 'stl', 'tle', 'les', 'ess', 'ss...","defaultdict(<class 'int'>, {'r': 1, 'a': 2, 'u...","['co', 'os', 'st', 'tl', 'le', 'es', 'ss', 'sa...",NaN,costlessautoparts,0.0,-1.0,1,NaN
498963,NaN,NaN,1397,NaN,b'www.costlessautoparts.com.au.',NaN,NaN,NaN,NaN,0,0,NaN,-1.0,1,NaN,21,parts,0,-1.0,"['c', 'o', 's', 't', 'l', 'e', 's', 's', 'a', ...",0,3.1588844500594413,0,com.au,0,0,NaN,0.0,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['cos', 'ost', 'stl', 'tle', 'les', 'ess', 'ss...","defaultdict(<class 'int'>, {'r': 1, 'a': 2, 'u...","['co', 'os', 'st', 'tl', 'le', 'es', 'ss', 'sa...",NaN,costlessautoparts,0.0,-1.0,1,NaN
498980,NaN,NaN,3599,NaN,b'www.neojeans.com.ar.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,12,jeans,0,-1.0,"['n', 'e', 'o', 'j', 'e', 'a', 'n', 's']",0,2.556

#### 'TTL'

In [23]:
non_digit_TTL_list = []
for value in list(data['TTL'].unique()):
    if pd.isna(value):
        continue
    elif str(value).isdigit():
        continue
    else:
        non_digit_TTL_list.append(value)

In [24]:
data.sample(15)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
349287,US,32475.0,14399,69.175.87.74,b'unifiedmediaclub.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,NaN,17,unified,0,-1.0,"['u', 'n', 'i', 'f', 'i', 'e', 'd', 'm', 'e', ...",0,3.2143834092848222,NaN,com,0,0,NaN,0.0,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['uni', 'nif', 'ifi', 'fie', 'ied', 'edm', 'dm...","defaultdict(<class 'int'>, {'e': 2, 'u': 2, 'i...","['un', 'ni', 'if', 'fi', 'ie', 'ed', 'dm', 'me...",NaN,unifiedmediaclub,NaN,NaN,0,US
99536,CN,134764.0,599,124.172.154.244,b'www.gzyuqun.com.',guang dong,NaN,CN,2014-12-07 13:37:15,0,0,"['GZYUQUN.COM', 'gzyuqun.com']",-1.0,1,NaN,11,un,0,-1.0,"['g', 'z', 'y', 'u', 'q', 'u', 'n']",0,2.5308571531099555,1965 days 22:22:03.483467,com,0,0,DomainAbuse@service.aliyun.com,0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['gzy', 'zyu', 'yuq', 'uqu', 'qun']","defaultdict(<class 'int'>, {'u': 2, 'q': 1, 'n...","['gz', 'zy', 'yu', 'uq', 'qu', 'un']",Alibaba Cloud Computing (Beijing) Co.,Ltd.,gzyuqun,2.0,0,CN
488806,US,13335.0,299,104.24.119.16,b'mlove.ru.',NaN,NaN,NaN,2004-01-21 21:00:00,0,0,MLOVE.RU,1407388.0,0,NaN,6,ove,0,-1.0,"['m', 'l', 'o', 'v', 'e']",0,2.15413541726763,5945 days 0:58:54.191003,ru,0,0,NaN,0.0,0,"[('vkontakte.ru', 86), ('ok.ru', 86)]",0,"['mlo', 'lov', 'ove']","defaultdict(<class 'int'>, {'m': 1, 'e': 1, 'o...","['ml', 'lo', 'ov', 've']",RU-CENTER-RU,mlove,2.0,-1.0,0,US
351597,NaN,NaN,7199,NaN,b'www.finjan.com.',NaN,NaN,NaN,1996-03-07 05:00:00,0,0,FINJAN.COM,615724.0,1,NaN,10,jan,0,-1.0,"['f', 'i', 'n', 'j', 'a', 'n']",0,2.31424653518228,8819 days 13:05:01.497759,com,0,0,abuse@web.com,0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['fin', 'inj', 'nja', 'jan']","defaultdict(<class 'int'>, {'i': 1, 'n': 2, 'a...","['fi', 'in', 'nj', 'ja', 'an']",Network Solutions,LLC,finjan,2.0,0,NaN
57525,CN,59029.0,599,103.42.79.203,b'cnautonews.com.',bei jing,NaN,CN,2004-02-24 06:50:52,0,0,"['CNAUTONEWS.COM', 'cnautonews.com']",852346.0,0,NaN,11,news,0,-1.0,"['c', 'n', 'a', 'u', 't', 'o', 'n', 'e', 'w', ...",0,2.9631196533066344,5904 days 11:26:44.610755,com,0,0,DomainAbuse@service.aliyun.com,0.0,0,"[('apple.com', 86), ('adobe.com', 86)]",0,"['cna', 'nau', 'aut', 'uto', 'ton', 'one', 'ne...","defaultdict(<class 'int'>, {'t': 1, 'u': 1, 'o...","['cn', 'na', 'au', 'ut', 'to', 'on', 'ne', 'ew...",Alibaba Cloud Computing (Beijing) Co.,Ltd.,cnautonews,4.0,0,CN
355611,TR,34619.0,3599,94.73.151.148,b'www.cayyolutesisat.net.',NaN,NaN,NaN,2014-05-12 20:25:44,0,0,CAYYOLUTESISAT.NET,-1.0,1,NaN,18,lutes,0,-1.0,"['c', 'a', 'y', 'y', 'o', 'l', 'u', 't', 'e', ...",0,3.2296576401308803,2178 days 23:00:26.392313,net,0,0,abuse@isimtescil.net,0.0,0,"[('cpanel.net', 86), ('behance.net', 86)]",0,"['cay', 'ayy', 'yyo', 'yol', 'olu', 'lut', 'ut...","defaultdict(<class 'int'>, {'t': 2, 'u': 1, 'i...","['ca', 'ay', 'yy', 'yo', 'ol', 'lu', 'ut', 'te...",FBS Inc.,cayyolutesisat,2.0,-1.0,0,TR
233385,US,13335.0,299,104.23.130.65,b'www.e-radio.gr.',NaN,NaN,NaN,NaN,0,0,NaN,9385.0,1,NaN,11,o,0,-1.0,"['e', '-', 'r', 'a', 'd', 'i', 'o']",0,2.7126753349281363,0,gr,0,0,NaN,0.0,0,"[('tripadvisor.com', 54), ('ok.ru', 54)]",0,"['e-r', '-ra', 'rad', 'adi', 'dio']","defaultdict(<class 'int'>, {'e': 1, '-': 1, 'o...","['e-', '-r', 'ra', 'ad', 'di', 'io']",NaN,e-radio,0.0,-1.0,0,US
327394,US,46606.0,14399,192.232.198.107,b'amandra.mx.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,NaN,8,and,0,-1.0,"['a', 'm', 'a', 'n', 'd', 'r', 'a']",0,2.0306390622295662,NaN,mx,0,0,NaN,0.0,0,"[('amazonaws.com', 61), ('amazon.com', 60)]",0,"['ama', 'man', 'and', 'ndr', 'dra']","defaultdict(<class 'int'>, {'m': 1, 'r': 1, 'a...","['am', 'ma', 'an', 'nd', 'dr', 'ra']",NaN,amandra,NaN,NaN,0,US
99104,US

In [25]:
data[data['TTL'].isin(non_digit_TTL_list)]
#data_TTL

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
494323,132422,64021.0,b'n85853.cn.',HK,150.129.42.39,NaN,NaN,NaN,2010-12-01 22:42:40,0,0,n85853.cn,-1.0,0,NaN,7,3,0,-1.0,"['n', '8', '5', '8', '5', '3']",0,1.8348756474779464,3428 days 2:51:32.789665,cn,0,0,da@gz.com,45.45454545454545,0,"[('miitbeian.gov.cn', 86), ('miibeian.gov.cn',...",0,"['n85', '858', '585', '853']","defaultdict(<class 'int'>, {'8': 2, '5': 2, 'n...","['n8', '85', '58', '85', '53']",广州名扬信息科技有限公司,n85853,4.0,-1.0,1,132422
494401,27411,15003.0,b'cooper.mylftv.com.',US,23.19.122.212,florida,NaN,US,2005-06-24 01:26:13,0,0,MYLFTV.COM,8178420.0,1,changeip.com,13,tv,0,-1.0,"['m', 'y', 'l', 'f', 't', 'v']",0,3.2619443552071625,5415 days 0:10:24.293324,com,0,0,"['abuse-contact@publicdomainregistry.com', 'ab...",0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['myl', 'ylf', 'lft', 'ftv']","defaultdict(<class 'int'>, {'v': 1, 'y': 1, 'e...","['my', 'yl', 'lf', 'ft', 'tv']",PDR Ltd. d/b/a PublicDomainRegistry.com,mylftv,10.0,-1.0,1,US
494555,133119,9308.0,b'amino-cn.com.',CN,120.133.19.118,REDACTED FOR PRIVACY,NaN,REDACTED FOR PRIVACY,2005-05-09 03:53:25,0,0,"['AMINO-CN.COM', 'amino-cn.com']",-1.0,0,REDACTED FOR PRIVACY,9,amino,0,-1.0,"['a', 'm', 'i', 'n', 'o', '-', 'c', 'n']",0,2.5954888901709436,5460 days 21:47:12.461152,com,0,0,supervision@xinnet.com,0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['ami', 'min', 'ino', 'no-', 'o-c', '-cn']","defaultdict(<class 'int'>, {'-': 1, 'n': 2, 'c...","['am', 'mi', 'in', 'no', 'o-', '-c', 'cn']",XINNET TECHNOLOGY CORPORATION,amino-cn,4.0,-1.0,1,133119
494648,133905,133380.0,b'18dd.net.',HK,113.52.134.81,Panama,NaN,PA,2019-10-07 03:45:15,0,0,"['18DD.NET', '18dd.net']",-1.0,0,WhoisGuard inc.,5,d,0,-1.0,"['1', '8', 'd', 'd']",0,1.4575424759098898,196 days 21:57:29.776325,net,0,0,"['abuse@namecheap.com', '7418ae19381641be88caf...",20.0,0,"[('slideshare.net', 86), ('doubleclick.net', 86)]",0,"['18d', '8dd']","defaultdict(<class 'int'>, {'1': 1, '8': 1, 'd...","['18', '8d', 'dd']",NAMECHEAP INC,18dd,4.0,-1.0,1,PA
494650,133905,133380.0,b'www.18dd.net.',HK,113.52.134.81,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,8,d,0,-1.0,"['1', '8', 'd', 'd']",0,1.7806390622295662,NaN,net,0,0,NaN,14.285714285714285,0,"[('slideshare.net', 86), ('doubleclick.net', 86)]",0,"['18d', '8dd']","defaultdict(<class 'int'>, {'1': 1, 'w': 3, '8...","['18', '8d', 'dd']",NaN,18dd,NaN,NaN,1,133905
494771,132422,64021.0,b'fffddd11.cn.',HK,150.129.42.39,NaN,NaN,NaN,2016-06-20 21:54:28,0,0,fffddd11.cn,-1.0,0,NaN,9,ddd,0,-1.0,"['f', 'f', 'f', 'd', 'd', 'd', '1', '1']",0,1.5388472230235068,1400 days 3:51:29.077528,cn,0,0,da@yyholdings.com,15.384615384615385,0,"[('4.cn', 86), ('360.cn', 86)]",0,"['fff', 'ffd', 'fdd', 'ddd', 'dd1', 'd11']","defaultdict(<class 'int'>, {'1': 2, 'd': 3, 'f...","['ff', 'ff', 'fd', 'dd', 'dd', 'd1', '11']",广州名扬信息科技有限公司,fffddd11,4.0,-1.0,1,132422
494926,23724,4808.0,b'ailvgo.com.',CN,103.235.227.58,REDACTED FOR PRIVACY,NaN,REDACTED FOR PRIVACY,2014-08-11 09:20:08,0,0,"['AILVGO.COM', 'ailvgo.com']",-1.0,0,REDACTED FOR PRIVACY,7,go,0,-1.0,"['a', 'i', 'l', 'v', 'g', 'o']",0,2.4063042189065182,2079 days 16:29:45.806180,com,0,0,supervision@xinnet.com,0.0,0,"[('go.com', 90), ('qq.com', 86)]",0,"['ail', 'ilv', 'lvg', 'vgo']","defaultdict(<class 'int'>, {'v': 1, 'g': 1, 'i...","['ai', 'il', 'lv', 'vg', 'go']",XINNET TECHNOLOGY CORPORATION,ailvgo,4.0,-1.0,1,23724
494975,9802,4847.0,b'hdxxpp.com.',CN,120.133.3.240,REDACTED FOR PRIVACY,NaN,REDACTED FOR PRIVACY,2009-09-10 08:35:14,0,0,"['HDXXPP.COM', 'hdxxpp.com']",-1.0,0,REDACTED FOR PRIVACY,7,pp,0,-1.0,"['h', 'd', 'x', 'x', 'p', 'p']",0,1.8348756474779464,3875 days 17:16:41.367259,com,0,0,supervision@xin

In [26]:
map_columns = {'Country' : 'TTL',
               'TTL' : 'Domain',
               'IP' : 'CountryCODE', 
               'Domain' : 'IP', 
               'CountryCODE' : 'Country'}

In [27]:
data_part = data[~data['TTL'].isin(non_digit_TTL_list)]
data_TTL = data[data['TTL'].isin(non_digit_TTL_list)]

In [28]:
data_TTL.rename(columns=map_columns, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\3954434219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_TTL.rename(columns=map_columns, inplace=True)


In [29]:
data = pd.concat([data_part, data_TTL], axis=0)
data

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
0,US,15169.0,46,172.217.1.174,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
1,NaN,NaN,299,NaN,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
2,NaN,NaN,38,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0,0,NaN
3,US,15169.0,299,172.217.164.228,b'www.google.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,NaN,com,0,0,NaN,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",NaN,google,NaN,NaN,0,US
4,NaN,NaN,96,NaN,b'facebook.com.',CA,NaN,US,1997-03-29 05:00:00,0,0,FACEBOOK.COM,4.0,0,Facebook inc.,9,facebook,0.0,-1.0,"['f', 'a', 'c', 'e', 'b', 'o', 'o', 'k']",0.0,2.5954888901709436,8426 days 10:53:52.505111,com,0,0,"['abusecomplaints@registrarsafe.com', 'domain@...",0.0,0,"[('facebook.com', 100), ('qq.com', 86)]",0,"['fac', 'ace', 'ceb', 'ebo', 'boo', 'ook']","defaultdict(<class 'int'>, {'e': 1, 'c': 1, 'o...","['fa', 'ac', 'ce', 'eb', 'bo', 'oo', 'ok']",RegistrarSafe LLC,facebook,4.0,-1.0,0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495917,BR,10429.0,18881,179.178.55.72,b'fm.3tn.com.br.',NaN,NaN,BR,NaN,0,0,NaN,-1.0,1,NaN,6,n,0,-1.0,"['3', 't', 'n']",0,2.15413541726763,0,com.br,0,0,NaN,6.666666666666668,0,"[('qq.com', 86), ('vk.com', 86)]",0,['3tn'],"defaultdict(<class 'int'>, {'t': 1, 'n': 1, '3...","['3t', 'tn']",NaN,3tn,0.0,-1.0,1,BR
496125,23724,4808.0,23724,117.50.115.191,b'yctuoyu.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,NaN,8,yu,0,-1.0,"['y', 'c', 't', 'u', 'o', 'y', 'u']",0,2.125,NaN,com,0,0,NaN,0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['yct', 'ctu', 'tuo', 'uoy', 'oyu']","defaultdict(<class 'int'>, {'t': 1, 'u': 2, 'y...","['yc', 'ct', 'tu', 'uo', 'oy', 'yu']",NaN,yctuoyu,NaN,NaN,1,CN
497049,38525,9340.0,38525,117.54.5.10,b'alphasite.indonet.co.id.',NaN,NaN,NaN,2001-06-02 13:33:31,0,0,indonet.co.id,3795366.0,1,NaN,17,indo,0,-1.0,"['i', 'n', 'd', 'o', 'n', 'e', 't']",0,3.2587885564709085,6897 days 13:13:12.941860,co.id,0,0,NaN,0.0,0,"[('t.co', 90), ('g.co', 86)]",0,"['ind', 'ndo', 'don', 'one', 'net']","defaultdict(<class 'int'>, {'d': 1, 'n': 2, 'e...","['in', 'nd', 'do', 'on', 'ne', 'et']",NaN,indonet,2.0,-1.0,1,ID
497574,23724,4808.0,23724,117.50.124.90,b'andq888.com

In [30]:
data['TTL'] = data['TTL'].astype(float)

In [31]:
data[(data['TTL'].isna())]

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE


#### 'Domain'

In [32]:
data['Domain'].describe()

count                      499134
unique                     313538
top       b'www.hugedomains.com.'
freq                          502
Name: Domain, dtype: object

In [33]:
set(data.Domain.str.isnumeric())

{False, True}

In [34]:
for idx, row in data.iterrows():
    if row['Domain'].isdigit():
        print(idx, row['Domain'])

495373 397220
495374 397220


In [35]:
data[data['Domain'] == '397220']

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
495373,397215,397213.0,397224.0,397218,397220,NSW,NaN,AU,2019-05-30 23:54:31,0,0,"['QANTASAIRWAYS.NET', 'qantasairways.net']",-1.0,1,QANTAS AIRWAYS LIMITED,17,airways,0,-1.0,"['q', 'a', 'n', 't', 'a', 's', 'a', 'i', 'r', ...",0,2.7882003211767907,326 days 2:07:26.409149,net,0,0,"['domainabuse@cscglobal.com', 'qfdnsadmin@qant...",0.0,0,"[('cpanel.net', 86), ('behance.net', 86)]",0,"['qan', 'ant', 'nta', 'tas', 'asa', 'sai', 'ai...","defaultdict(<class 'int'>, {'y': 1, 'n': 1, 'r...","['qa', 'an', 'nt', 'ta', 'as', 'sa', 'ai', 'ir...",CSC CORPORATE DOMAINS inc.,qantasairways,12.0,-1.0,1,AU
495374,397215,397213.0,397224.0,397218,397220,NSW,NaN,AU,1996-05-17 04:00:00,0,0,"['QANTAS.COM', 'qantas.com']",6599.0,0,QANTAS AIRWAYS LIMITED,7,qantas,0,-1.0,"['q', 'a', 'n', 't', 'a', 's']",0,2.1205899331922318,8739 days 22:01:58.217832,com,0,0,"['domainabuse@cscglobal.com', 'qfdnsadmin@qant...",0.0,0,"[('quantcast.com', 87), ('qq.com', 86)]",0,"['qan', 'ant', 'nta', 'tas']","defaultdict(<class 'int'>, {'t': 1, 's': 1, 'n...","['qa', 'an', 'nt', 'ta', 'as']",CSC CORPORATE DOMAINS inc.,qantas,12.0,-1.0,1,AU


In [36]:
for idx in list(data[data['Domain'] == '397220'].index):
    data.loc[idx, 'Domain'] = data.iloc[idx]['Domain_Name'][1]

In [37]:
data['Domain'] = data['Domain'].astype('unicode')

In [38]:
set(data.Domain.str.isnumeric())

{False}

#### 'shortened'

In [39]:
data['shortened'].describe()

count     497789
unique        97
top         -1.0
freq      449182
Name: shortened, dtype: object

In [40]:
data['shortened'] = data['shortened'].astype(str)

In [41]:
data['shortened'].value_counts(normalize=True)

shortened
-1.0                                                                                                                                                                     0.935114
0                                                                                                                                                                        0.059050
0.0                                                                                                                                                                      0.002781
nan                                                                                                                                                                      0.002695
['t', 'a', 'x', 'a', 'd', 'm', 'i', 'n']                                                                                                                                 0.000016
                                                                                                    

In [42]:
data[data['shortened'] == '-1.0'].sample(2)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
404103,NaN,NaN,299.0,NaN,b'www.nationen.no.',NaN,NaN,NaN,NaN,0,0,NaN,466174.0,1,NaN,12,nation,0,-1.0,"['n', 'a', 't', 'i', 'o', 'n', 'e', 'n']",0,2.4937343753004817,0,no,0,0,NaN,0.0,0,"[('nationalgeographic.com', 66), ('one.com', 64)]",0,"['nat', 'ati', 'tio', 'ion', 'one', 'nen']","defaultdict(<class 'int'>, {'t': 1, 'e': 1, 'i...","['na', 'at', 'ti', 'io', 'on', 'ne', 'en']",NaN,nationen,0.0,-1.0,0,NaN
86800,US,16509.0,299.0,52.74.138.163,b'jagranjosh.com.',Delhi,NaN,IN,2009-09-11 11:14:02,0,0,"['JAGRANJOSH.COM', 'jagranjosh.com']",1369.0,0,MMI Online Limited,11,josh,0,-1.0,"['j', 'a', 'g', 'r', 'a', 'n', 'j', 'o', 's', ...",0,2.7813014714884523,3878 days 19:08:08.118657,com,0,0,abuse@godaddy.com,0.0,0,"[('apple.com', 86), ('adobe.com', 86)]",0,"['jag', 'agr', 'gra', 'ran', 'anj', 'njo', 'jo...","defaultdict(<class 'int'>, {'h': 1, 'o': 1, 'n...","['ja', 'ag', 'gr', 'ra', 'an', 'nj', 'jo', 'os...",GoDaddy.com LLC,jagranjosh,3.0,-1.0,0,US


In [43]:
data[(data['shortened'] != '-1.0') &(data['longest_word'] ==  '-1.0')].head(2)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
101824,RU,48287.0,3599.0,212.192.194.2,b'adminerm.ru.',NaN,NaN,NaN,2008-02-18 21:00:00,0,0AdminERM.RU,3492859.0,0,MUP Administratsiya Ermakovskogo rayona,9Admin,0,-1.0,"['a', 'd', 'm', 'i', 'n', 'e', 'r', 'm']",0,2.5954888901709436,4449 days 16:07:45.599266,ru,0,0,NaN,0.0,0,"[('mail.ru', 86), ('ok.ru', 86)]",0,"['adm', 'dmi', 'min', 'ine', 'ner', 'erm']","defaultdict(<class 'int'>, {'e': 1, 'i': 1, 'd...","['ad', 'dm', 'mi', 'in', 'ne', 'er', 'rm']",DOMENUS-RUAdminerm,2.0,-1.0,NaN,NaN,NaN,0,RU
128145,NL,14061.0,29.0,188.166.77.72,b'adminico.nl.',NaN,NaN,Updated Date: 2020-01-08,NaN,0,0Adminico.nl,-1.0,0,NaN,9Admin,0,-1.0,"['a', 'd', 'm', 'i', 'n', 'i', 'c', 'o']",0,2.5954888901709436,0,nl,0,0,NaN,0.0,0,"[('t.co', 64), ('g.co', 64)]",0,"['adm', 'dmi', 'min', 'ini', 'nic', 'ico']","defaultdict(<class 'int'>, {'i': 2, 'o': 1, 'd...","['ad', 'dm', 'mi', 'in', 'ni', 'ic', 'co']",nanAdminico,0.0,-1.0,NaN,NaN,NaN,0,NL


In [44]:
temp_df_rest = data[(data['shortened'] == '-1.0') | (data['longest_word'] != '-1.0')]

In [45]:
temp_df = data[(data['shortened'] != '-1.0') & (data['longest_word'] == '-1.0')]

In [46]:
len(data), len(temp_df)+len(temp_df_rest)

(499134, 499134)

In [48]:
temp_df['Organization_int'] = temp_df['Organization'].apply(lambda x: re.match(r'[0-9]+', x)[0])
temp_df['Organization_str'] = temp_df['Organization'].apply(lambda x: re.sub(r'[0-9]+', '', x))

temp_df['hex_8_int'] = temp_df['hex_8'].apply(lambda x: x[0] )
temp_df['hex_8_str'] = temp_df['hex_8'].apply(lambda x: x[1:] )

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\1386969683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Organization_int'] = temp_df['Organization'].apply(lambda x: re.match(r'[0-9]+', x)[0])
C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\1386969683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Organization_str'] = temp_df['Organization'].apply(lambda x: re.sub(r'[0-9]+', '', x))
C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\1386969683.py:4: SettingWithCopyWarning: 
A value

In [49]:
temp_df = temp_df.drop(['hex_8', 'Organization', 'Name_Server_Count', 'Page_Rank'], axis=1)

In [50]:
map_col_names = {
 'hex_8_int': 'hex_8',
 'hex_8_str': 'Domain_Name', 
 'Domain_Name' : 'Alexa_Rank', 
 'Alexa_Rank' : 'subdomain', 
 'subdomain' : 'Organization', 
 'Organization_int': 'len', 
 'Organization_str' : 'longest_word', 
 'len' : 'oc_32', 
 'longest_word' : 'shortened', 
 'oc_32' : '1gram',
 'shortened': 'obfuscate_at_sign', 
 '1gram' : 'entropy', 
 'obfuscate_at_sign' : 'Domain_Age', 
 'entropy' : 'tld', 
 'Domain_Age': 'dec_8', 
 'tld':'dec_32', 
 'dec_8': 'Emails',
 'dec_32': 'numeric_percentage', 
 'Emails':'puny_coded', 
 'numeric_percentage':'typos',
 'puny_coded': 'oc_8', 
 'typos': '3gram'
 ,'oc_8': 'char_distribution', 
 '3gram': '2gram',
 'char_distribution':'Registrar',
 '2gram':'Name_Server_Count',
 'Registrar':'Page_Rank'
}


In [51]:
temp_df.rename(columns=map_col_names, inplace=True)

In [52]:
data = pd.concat([temp_df_rest, temp_df], axis=0)

In [53]:
data

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
0,US,15169.0,46.0,172.217.1.174,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
1,NaN,NaN,299.0,NaN,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
2,NaN,NaN,38.0,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0,0,NaN
3,US,15169.0,299.0,172.217.164.228,b'www.google.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,NaN,com,0,0,NaN,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",NaN,google,NaN,NaN,0,US
4,NaN,NaN,96.0,NaN,b'facebook.com.',CA,NaN,US,1997-03-29 05:00:00,0,0,FACEBOOK.COM,4.0,0,Facebook inc.,9,facebook,0.0,-1.0,"['f', 'a', 'c', 'e', 'b', 'o', 'o', 'k']",0.0,2.5954888901709436,8426 days 10:53:52.505111,com,0,0,"['abusecomplaints@registrarsafe.com', 'domain@...",0.0,0,"[('facebook.com', 100), ('qq.com', 86)]",0,"['fac', 'ace', 'ceb', 'ebo', 'boo', 'ook']","defaultdict(<class 'int'>, {'e': 1, 'c': 1, 'o...","['fa', 'ac', 'ce', 'eb', 'bo', 'oo', 'ok']",RegistrarSafe LLC,facebook,4.0,-1.0,0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412303,RU,12389.0,21599.0,87.226.148.36,b'admin-smolensk.ru.',NaN,NaN,NaN,2009-09-09 20:00:00,0,0,Admin-SMOLENSK.RU,38489.0,0,Department Cmolenskoy field of information tec...,15,Admin,0,-1.0,"['a', 'd', 'm', 'i', 'n', '-', 's', 'm', 'o', ...",0,3.246431222567951,3885 days 20:36:51.033131,ru,0,0,NaN,0.0,0,"[('yandex.ru', 86), ('mail.ru', 86)]",0,"['adm', 'dmi', 'min', 'in-', 'n-s', '-sm', 'sm...","defaultdict(<class 'int'>, {'e': 1, 'i': 1, 'o...","['ad', 'dm', 'mi', 'in', 'n-', '-s', 'sm', 'mo...",REGRU-RUAdmin-smolensk,NaN,2.0,-1.0,0,RU
412304,RU,12389.0,18990.0,87.226.148.36,b'www.admin-smolensk.ru.',NaN,NaN,NaN,2009-09-09 20:00:00,0,0,Admin-SMOLENSK.RU,38489.0,1,Department Cmolenskoy field of information tec...,18,Admin,0,-1.0,"['a', 'd', 'm', 'i', 'n', '-', 's', 'm', 'o', ...",0,3.3407687512419915,3885 days 20:36:51.981985,ru,0,0,NaN,0.0,0,"[('yandex.ru', 86), ('mail.ru', 86)]",0,"['adm', 'dmi', 'min', 'in-', 'n-s', '-sm', 'sm...","defaultdict(<class 'int'>, {'e': 1, '-': 1, 'o...","['ad', 'dm', 'mi', 'in', 'n-', '-s', 'sm', 'mo...",REGRU-RUAdmin-smolensk,NaN,2.0,-1.0,0,RU
412305,NaN,NaN,18990.0,NaN,b'www.admin-smolensk.ru.',NaN,Na

In [54]:
temp_data_complement = data[~data['shortened'].isin(['0', '0.0'])]

In [55]:
temp_data = data[data['shortened'].isin(['0', '0.0'])]

In [56]:
temp_data['Organization'] = temp_data.apply(lambda x: str(x.Organization)+str(x.len) if not pd.isna(x.len) else x.Organization, axis=1)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\1770137636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['Organization'] = temp_data.apply(lambda x: str(x.Organization)+str(x.len) if not pd.isna(x.len) else x.Organization, axis=1)


In [57]:
temp_data_complement.head(3)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
0,US,15169.0,46.0,172.217.1.174,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
1,NaN,NaN,299.0,NaN,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
2,NaN,NaN,38.0,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0,0,NaN


In [58]:
temp_data.head(3)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
16407,US,16509.0,44.0,54.69.86.108,b'tapas.io.',NaN,Domains By Proxy,LLC,NaN,2015-08-31 21:00:28,0,0,TAPAS.IO,12456.0,0,NaN,6,tapas,0,-1.0,"['t', 'a', 'p', 'a', 's']",0,1.8208020839342969,1697 days 2:51:49.990193,io,0,0,NaN,0.0,0,"[('t.co', 64), ('aboutads.info', 57)]",0,"['tap', 'apa', 'pas']","defaultdict(<class 'int'>, {'t': 1, 'a': 2, 'p...","['ta', 'ap', 'pa', 'as']",GoDaddy.com LLC,tapas,4.0,0,US
16408,US,16509.0,59.0,54.69.86.108,b'tapas.io.',NaN,Domains By Proxy,LLC,NaN,2015-08-31 21:00:28,0,0,TAPAS.IO,12456.0,0,NaN,6,tapas,0,-1.0,"['t', 'a', 'p', 'a', 's']",0,1.8208020839342969,1697 days 2:51:54.552533,io,0,0,NaN,0.0,0,"[('t.co', 64), ('aboutads.info', 57)]",0,"['tap', 'apa', 'pas']","defaultdict(<class 'int'>, {'t': 1, 'a': 2, 'p...","['ta', 'ap', 'pa', 'as']",GoDaddy.com LLC,tapas,4.0,0,US
16580,397215,397213.0,299.0,204.74.99.100,b'beachbody.com.',CA,NaN,US,1999-08-17 16:39:35,0,0,"['BEACHBODY.COM', 'beachbody.com']",21362.0,0,Beachbody LLC,LLC,10,beach,0,-1.0,"['b', 'e', 'a', 'c', 'h', 'b', 'o', 'd', 'y']",0,2.789735285398626,7555 days 7:17:46.568410,com,0,0,"['domainabuse@cscglobal.com', 'domain_managers...",0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['bea', 'eac', 'ach', 'chb', 'hbo', 'bod', 'ody']","defaultdict(<class 'int'>, {'e': 1, 'h': 1, 'o...","['be', 'ea', 'ac', 'ch', 'hb', 'bo', 'od', 'dy']",CSC CORPORATE DOMAINS inc.,beachbody,8.0,0,US


In [59]:
map_values = {
'longest_word':'len', 
'oc_32' : 'longest_word', 
'shortened' : 'oc_32', 
'1gram' : 'shortened', 
'obfuscate_at_sign' : '1gram', 
'entropy' : 'obfuscate_at_sign', 
'Domain_Age' : 'entropy', 
'tld' : 'Domain_Age', 
'dec_8' : 'tld', 
'dec_32' : 'dec_8',
'Emails' : 'dec_32', 
'numeric_percentage' : 'Emails', 
'puny_coded' :'numeric_percentage', 
'typos': 'puny_coded', 
'oc_8' : 'typos', 
'3gram' : 'oc_8', 
'char_distribution' : '3gram', 
'2gram' : 'char_distribution', 
'Registrar' : '2gram', 
'sld': 'Registrar', 
'Name_Server_Count': 'sld', 
'Page_Rank': 'Name_Server_Count', 
'len' : 'Page_Rank' 
}

In [60]:
temp_data.rename(columns=map_values, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\2757519069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.rename(columns=map_values, inplace=True)


In [61]:
len(temp_data_complement), len(temp_data), len(temp_data_complement) + len(temp_data)

(468283, 30851, 499134)

In [62]:
data = pd.concat([temp_data_complement, temp_data], axis=0)

In [63]:
data

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
0,US,15169.0,46.0,172.217.1.174,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
1,NaN,NaN,299.0,NaN,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
2,NaN,NaN,38.0,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0,0,NaN
3,US,15169.0,299.0,172.217.164.228,b'www.google.com.',NaN,NaN,NaN,NaN,0,0,NaN,NaN,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,NaN,com,0,0,NaN,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",NaN,google,NaN,NaN,0,US
4,NaN,NaN,96.0,NaN,b'facebook.com.',CA,NaN,US,1997-03-29 05:00:00,0,0,FACEBOOK.COM,4.0,0,Facebook inc.,9,facebook,0.0,-1.0,"['f', 'a', 'c', 'e', 'b', 'o', 'o', 'k']",0.0,2.5954888901709436,8426 days 10:53:52.505111,com,0,0,"['abusecomplaints@registrarsafe.com', 'domain@...",0.0,0,"[('facebook.com', 100), ('qq.com', 86)]",0,"['fac', 'ace', 'ceb', 'ebo', 'boo', 'ook']","defaultdict(<class 'int'>, {'e': 1, 'c': 1, 'o...","['fa', 'ac', 'ce', 'eb', 'bo', 'oo', 'ok']",RegistrarSafe LLC,facebook,4.0,-1.0,0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494024,US,36351.0,21599.0,174.37.62.109,b'camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,0.0,Domains By Proxy LLC,8,camaro,0.0,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.375,4785 days 18:22:20.057263,com,0,0,"['abuse@godaddy.com', 'CAMARO6.COM@domainsbypr...",7.692308,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['cam', 'ama', 'mar', 'aro', 'ro6']","defaultdict(<class 'int'>, {'o': 1, 'r': 1, 'm...","['ca', 'am', 'ma', 'ar', 'ro', 'o6']",GoDaddy.com LLC,camaro6,2.0,LLC,0,US
494025,US,36351.0,21599.0,174.37.62.109,b'www.camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,1.0,Domains By Proxy LLC,11,camaro,0.0,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.530857153109955,4785 days 18:22:21.038080,com,0,0,"['abuse@godaddy.com', 'CAMARO6.COM@domainsbypr...",5.882353,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['cam', 'ama', 'mar', 'aro', 'ro6']","defaultdict(<class 'int'>, {'o': 1, 'r': 1, 'm...","['ca', 'am', 'ma', 'ar', 'ro', 'o6']",GoDaddy.com LLC,camaro6,2.0,LLC,0,US
494026,US,36351.0,21599.0,174.37.62.109,b'www.camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,1.0

#### 'dec_8'

In [64]:
data['dec_8'].describe()

count     499134
unique        12
top            0
freq      482511
Name: dec_8, dtype: object

In [65]:
data_temp_complement =  data[data['dec_8'].isin(['0', 0, '0.0'])]
data_temp =  data[~data['dec_8'].isin(['0', 0, '0.0'])]

In [66]:
data_temp_complement.head(3)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
0,US,15169.0,46.0,172.217.1.174,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:49.631067,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
1,NaN,NaN,299.0,NaN,b'google.com.',CA,NaN,US,1997-09-15 04:00:00,0,0,"['GOOGLE.COM', 'google.com']",1.0,0,Google LLC,7,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,1.8348756474779464,8256 days 11:53:50.479085,com,0,0,"['abusecomplaints@markmonitor.com', 'whoisrequ...",0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'g...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor inc.,google,8.0,-1.0,0,US
2,NaN,NaN,38.0,NaN,b'www.google.com.',NaN,NaN,NaN,1997-09-15 04:00:00,0,0,GOOGLE.COM,1.0,1,NaN,10,google,0.0,-1.0,"['g', 'o', 'o', 'g', 'l', 'e']",0.0,2.1142465351822795,8256 days 11:53:51.264191,com,0,0,abusecomplaints@markmonitor.com,0.0,0,"[('google.com', 100), ('google.com.br', 95)]",0,"['goo', 'oog', 'ogl', 'gle']","defaultdict(<class 'int'>, {'e': 1, 'o': 2, 'w...","['go', 'oo', 'og', 'gl', 'le']",MarkMonitor Inc.,google,4.0,-1.0,0,NaN


In [67]:
data_temp.head(3)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,len,longest_word,oc_32,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,malware,CountryCODE
36867,US,16509.0,3599.0,54.72.9.51,b'bearevent.fr.',NaN,NaN,NaN,2020-04-23 06:21:17,0,0,bearevent.fr,-1.0,0,NaN,10,event,0,-1.0,"['b', 'e', 'a', 'r', 'e', 'v', 'e', 'n', 't']",0,2.51424653518228,1 day,3:04:47.460977,fr,0,0,"['admin@tldregistrarsolutions.com', 'nic@nic.fr']",0.0,0,"[('free.fr', 86), ('1and1.fr', 86)]",0,"['bea', 'ear', 'are', 'rev', 'eve', 'ven', 'ent']","defaultdict(<class 'int'>, {'e': 3, 't': 1, 'r...","['be', 'ea', 'ar', 're', 'ev', 've', 'en', 'nt']",TLD Registrar Solutions Ltd,bearevent,3.0,0,US
37017,US,45102.0,599.0,47.74.230.247,b'icoulee.com.',Wilayah Persekutuan,NaN,Malaysia,2020-04-22 18:48:37,0,0,"['ICOULEE.COM', 'icoulee.com']",5409080.0,0,Whoisprotection.cc,8,coulee,0,-1.0,"['i', 'c', 'o', 'u', 'l', 'e', 'e']",0,2.375,1 day,14:43:29.888168,com,0,0,"['compliance_abuse@webnic.cc', 'reg_18101676@w...",0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['ico', 'cou', 'oul', 'ule', 'lee']","defaultdict(<class 'int'>, {'e': 2, 'u': 1, 'i...","['ic', 'co', 'ou', 'ul', 'le', 'ee']",WEBCC,icoulee,2.0,0,US
53691,DE,24940.0,3599.0,88.99.194.75,b'stmaryspmukuru.org.',as6d4as64d6,NaN,TH,2020-04-24 17:33:53,0,0,STMARYSPMUKURU.ORG,-1.0,0,NaN,15,kuru,0,-1.0,"['s', 't', 'm', 'a', 'r', 'y', 's', 'p', 'm', ...",0,2.92943872242372,-1 day,23:06:04.083208,org,0,0,"['abuse@name.com', 'copyyworld@gmail.com']",0.0,0,"[('w3.org', 86), ('mozilla.org', 86)]",0,"['stm', 'tma', 'mar', 'ary', 'rys', 'ysp', 'sp...","defaultdict(<class 'int'>, {'t': 1, 'y': 1, 'p...","['st', 'tm', 'ma', 'ar', 'ry', 'ys', 'sp', 'pm...",Name.com inc.,stmaryspmukuru,8.0,0,DE


In [68]:
map_col_names = {
'oc_32' : 'longest_word', 
'1gram' : 'shortened', 
'obfuscate_at_sign' :  '1gram',
'entropy' : 'obfuscate_at_sign',
'Domain_Age' : 'entropy',
'tld': 'Domain_Age',
'dec_8': 'tld',
'dec_32' : 'dec_8',
'Emails': 'dec_32',
'numeric_percentage' : 'Emails',
'puny_coded' : 'numeric_percentage', 
'typos' : 'puny_coded' ,
'oc_8' : 'typos', 
'3gram' : 'oc_8', 
'char_distribution' : '3gram', 
'2gram' : 'char_distribution', 
'Registrar' : '2gram',
'Name_Server_Count' : 'Registrar',
'Page_Rank' : 'Name_Server_Count',
'sld' : 'oc_32',
'len' : 'Page_Rank',
'longest_word' : 'len',
'shortened' : 'sld'
}


In [69]:
data_temp.rename(columns=map_col_names, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\4257385343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_temp.rename(columns=map_col_names, inplace=True)


In [70]:
data = pd.concat([data_temp, data_temp_complement], axis=0)
data

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,oc_32,Registrar,Name_Server_Count,malware,CountryCODE
36867,US,16509.0,3599.0,54.72.9.51,b'bearevent.fr.',NaN,NaN,NaN,2020-04-23 06:21:17,0,0,bearevent.fr,-1.0,0,NaN,10,event,0,-1.0,"['b', 'e', 'a', 'r', 'e', 'v', 'e', 'n', 't']",0,2.51424653518228,1 day,3:04:47.460977,fr,0,0,"['admin@tldregistrarsolutions.com', 'nic@nic.fr']",0.0,0,"[('free.fr', 86), ('1and1.fr', 86)]",0,"['bea', 'ear', 'are', 'rev', 'eve', 'ven', 'ent']","defaultdict(<class 'int'>, {'e': 3, 't': 1, 'r...","['be', 'ea', 'ar', 're', 'ev', 've', 'en', 'nt']",TLD Registrar Solutions Ltd,bearevent,3.0,0,US
37017,US,45102.0,599.0,47.74.230.247,b'icoulee.com.',Wilayah Persekutuan,NaN,Malaysia,2020-04-22 18:48:37,0,0,"['ICOULEE.COM', 'icoulee.com']",5409080.0,0,Whoisprotection.cc,8,coulee,0,-1.0,"['i', 'c', 'o', 'u', 'l', 'e', 'e']",0,2.375,1 day,14:43:29.888168,com,0,0,"['compliance_abuse@webnic.cc', 'reg_18101676@w...",0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['ico', 'cou', 'oul', 'ule', 'lee']","defaultdict(<class 'int'>, {'e': 2, 'u': 1, 'i...","['ic', 'co', 'ou', 'ul', 'le', 'ee']",WEBCC,icoulee,2.0,0,US
53691,DE,24940.0,3599.0,88.99.194.75,b'stmaryspmukuru.org.',as6d4as64d6,NaN,TH,2020-04-24 17:33:53,0,0,STMARYSPMUKURU.ORG,-1.0,0,NaN,15,kuru,0,-1.0,"['s', 't', 'm', 'a', 'r', 'y', 's', 'p', 'm', ...",0,2.92943872242372,-1 day,23:06:04.083208,org,0,0,"['abuse@name.com', 'copyyworld@gmail.com']",0.0,0,"[('w3.org', 86), ('mozilla.org', 86)]",0,"['stm', 'tma', 'mar', 'ary', 'rys', 'ysp', 'sp...","defaultdict(<class 'int'>, {'t': 1, 'y': 1, 'p...","['st', 'tm', 'ma', 'ar', 'ry', 'ys', 'sp', 'pm...",Name.com inc.,stmaryspmukuru,8.0,0,DE
130027,US,13335.0,299.0,104.27.183.79,b'genericzithromax.com.',CO,NaN,US,2020-04-24 18:37:44,0,0,"['GENERICZITHROMAX.COM', 'GenericZithrOmax.com']",-1.0,0,NaN,17,generic,0,-1.0,"['g', 'e', 'n', 'e', 'r', 'i', 'c', 'z', 'i', ...",0,3.494082674117968,1 day,6:20:35.445595,com,0,0,"['abuse@NameBright.com', 'GenericZithrOmax.com...",0.0,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['gen', 'ene', 'ner', 'eri', 'ric', 'icz', 'cz...","defaultdict(<class 'int'>, {'e': 2, 't': 1, 'o...","['ge', 'en', 'ne', 'er', 'ri', 'ic', 'cz', 'zi...",DropCatch.com 865 LLC,genericzithromax,4.0,0,US
130028,NaN,NaN,299.0,NaN,b'genericzithromax.com.',CO,NaN,US,2020-04-24 18:37:44,0,0,"['GENERICZITHROMAX.COM', 'GenericZithrOmax.com']",-1.0,0,NaN,17,generic,0,-1.0,"['g', 'e', 'n', 'e', 'r', 'i', 'c', 'z', 'i', ...",0,3.494082674117968,1 day,6:20:36.431331,com,0,0,"['abuse@NameBright.com', 'GenericZithrOmax.com...",0.0,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['gen', 'ene', 'ner', 'eri', 'ric', 'icz', 'cz...","defaultdict(<class 'int'>, {'e': 2, 't': 1, 'o...","['ge', 'en', 'ne', 'er', 'ri', 'ic', 'cz', 'zi...",DropCatch.com 865 LLC,genericzithromax,4.0,0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494024,US,36351.0,21599.0,174.37.62.109,b'camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,0.0,Domains By Proxy LLC,LLC,8,camaro,camaro6,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.375,4785 days 18:22:20.057263,com,0,0,"['abuse@godaddy.com', 'CAMARO6.COM@domainsbypr...",7.692308,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['cam', 'ama', 'mar', 'aro', 'ro6']","defaultdict(<class 'int'>, {'o': 1, 'r': 1, 'm...","['ca', 'am', 'ma', 'ar', 'ro', 'o6']",0.0,GoDaddy.com LLC,2.0,0,US
494025,US,36351.0,21599.0,174.37.62.109,b'www.camaro6.com.',Arizona,NaN,US,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,1.0,Domains By Proxy LLC,LLC,11,camaro,camaro6,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.5308571

In [71]:
data['dec_8'] = data['dec_8'].astype(float)

#### 'longest_word'

In [72]:
data['longest_word'].describe()

count     498998
unique     34981
top       online
freq        2126
Name: longest_word, dtype: object

In [73]:
data['longest_word'] = data['longest_word'].astype(str)

In [74]:
data['longest_word'].unique()

array(['0', 'google', 'facebook', ..., 'nutrients', 'sleaze', 'nakata'],
      dtype=object)

In [75]:
idx_list = []
for idx, row in data.iterrows():
    if row.longest_word:
        try:
            int(row.longest_word)
        except:
            continue
        idx_list.append(row.longest_word)

In [76]:
data[data['longest_word'] == 'Admin'].sample(3)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,oc_32,Registrar,Name_Server_Count,malware,CountryCODE
340986,RU,12389.0,21599.0,87.226.148.36,b'www.admin-smolensk.ru.',NaN,NaN,NaN,2009-09-09 20:00:00,0,0,Admin-SMOLENSK.RU,38632.0,1,Department Cmolenskoy field of information tec...,-1.0,18,Admin,NaN,-1.0,"['a', 'd', 'm', 'i', 'n', '-', 's', 'm', 'o', ...",0,3.3407687512419915,3884 days 18:27:53.812159,ru,0.0,0,NaN,0.0,0,"[('yandex.ru', 86), ('mail.ru', 86)]",0,"['adm', 'dmi', 'min', 'in-', 'n-s', '-sm', 'sm...","defaultdict(<class 'int'>, {'e': 1, '-': 1, 'o...","['ad', 'dm', 'mi', 'in', 'n-', '-s', 'sm', 'mo...",0,REGRU-RUAdmin-smolensk,2.0,0,RU
412303,RU,12389.0,21599.0,87.226.148.36,b'admin-smolensk.ru.',NaN,NaN,NaN,2009-09-09 20:00:00,0,0,Admin-SMOLENSK.RU,38489.0,0,Department Cmolenskoy field of information tec...,-1.0,15,Admin,NaN,-1.0,"['a', 'd', 'm', 'i', 'n', '-', 's', 'm', 'o', ...",0,3.246431222567951,3885 days 20:36:51.033131,ru,0.0,0,NaN,0.0,0,"[('yandex.ru', 86), ('mail.ru', 86)]",0,"['adm', 'dmi', 'min', 'in-', 'n-s', '-sm', 'sm...","defaultdict(<class 'int'>, {'e': 1, 'i': 1, 'o...","['ad', 'dm', 'mi', 'in', 'n-', '-s', 'sm', 'mo...",0,REGRU-RUAdmin-smolensk,2.0,0,RU
196207,CA,13768.0,1403.0,69.90.160.76,b'adminwhiz.ca.',NaN,REDACTED FOR PRIVACY,NaN,2014-08-31 22:14:16,0,0,Adminwhiz.ca,-1.0,0,NaN,0.0,10,Admin,-1.0,-1.0,"['a', 'd', 'm', 'i', 'n', 'w', 'h', 'i', 'z']",0,2.789735285398626,2065 days 5:19:34.823533,ca,0.0,0,Please ask the Registrar of Record identified ...,Admin or Other contacts of the queried domain...,0.0,0,"[('ca.gov', 86), ('cbc.ca', 86)]",0,"['adm', 'dmi', 'min', 'inw', 'nwh', 'whi', 'hiz']","defaultdict(<class 'int'>, {'h': 1, 'i': 2, 'd...",0,"['ad', 'dm', 'mi', 'in', 'nw', 'wh', 'hi', 'iz']",nanAdminwhiz,0,CA


In [77]:
data[data['longest_word'].isin(set(idx_list))]

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,2gram,oc_32,Registrar,Name_Server_Count,malware,CountryCODE
36867,US,16509.0,3599.0,54.72.9.51,b'bearevent.fr.',NaN,NaN,NaN,2020-04-23 06:21:17,0,0,bearevent.fr,-1.0,0,NaN,10,event,0,-1.0,"['b', 'e', 'a', 'r', 'e', 'v', 'e', 'n', 't']",0,2.51424653518228,1 day,3:04:47.460977,fr,0.0,0,"['admin@tldregistrarsolutions.com', 'nic@nic.fr']",0.0,0,"[('free.fr', 86), ('1and1.fr', 86)]",0,"['bea', 'ear', 'are', 'rev', 'eve', 'ven', 'ent']","defaultdict(<class 'int'>, {'e': 3, 't': 1, 'r...","['be', 'ea', 'ar', 're', 'ev', 've', 'en', 'nt']",TLD Registrar Solutions Ltd,bearevent,3.0,0,US
37017,US,45102.0,599.0,47.74.230.247,b'icoulee.com.',Wilayah Persekutuan,NaN,Malaysia,2020-04-22 18:48:37,0,0,"['ICOULEE.COM', 'icoulee.com']",5409080.0,0,Whoisprotection.cc,8,coulee,0,-1.0,"['i', 'c', 'o', 'u', 'l', 'e', 'e']",0,2.375,1 day,14:43:29.888168,com,0.0,0,"['compliance_abuse@webnic.cc', 'reg_18101676@w...",0.0,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['ico', 'cou', 'oul', 'ule', 'lee']","defaultdict(<class 'int'>, {'e': 2, 'u': 1, 'i...","['ic', 'co', 'ou', 'ul', 'le', 'ee']",WEBCC,icoulee,2.0,0,US
53691,DE,24940.0,3599.0,88.99.194.75,b'stmaryspmukuru.org.',as6d4as64d6,NaN,TH,2020-04-24 17:33:53,0,0,STMARYSPMUKURU.ORG,-1.0,0,NaN,15,kuru,0,-1.0,"['s', 't', 'm', 'a', 'r', 'y', 's', 'p', 'm', ...",0,2.92943872242372,-1 day,23:06:04.083208,org,0.0,0,"['abuse@name.com', 'copyyworld@gmail.com']",0.0,0,"[('w3.org', 86), ('mozilla.org', 86)]",0,"['stm', 'tma', 'mar', 'ary', 'rys', 'ysp', 'sp...","defaultdict(<class 'int'>, {'t': 1, 'y': 1, 'p...","['st', 'tm', 'ma', 'ar', 'ry', 'ys', 'sp', 'pm...",Name.com inc.,stmaryspmukuru,8.0,0,DE
130027,US,13335.0,299.0,104.27.183.79,b'genericzithromax.com.',CO,NaN,US,2020-04-24 18:37:44,0,0,"['GENERICZITHROMAX.COM', 'GenericZithrOmax.com']",-1.0,0,NaN,17,generic,0,-1.0,"['g', 'e', 'n', 'e', 'r', 'i', 'c', 'z', 'i', ...",0,3.494082674117968,1 day,6:20:35.445595,com,0.0,0,"['abuse@NameBright.com', 'GenericZithrOmax.com...",0.0,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['gen', 'ene', 'ner', 'eri', 'ric', 'icz', 'cz...","defaultdict(<class 'int'>, {'e': 2, 't': 1, 'o...","['ge', 'en', 'ne', 'er', 'ri', 'ic', 'cz', 'zi...",DropCatch.com 865 LLC,genericzithromax,4.0,0,US
130028,NaN,NaN,299.0,NaN,b'genericzithromax.com.',CO,NaN,US,2020-04-24 18:37:44,0,0,"['GENERICZITHROMAX.COM', 'GenericZithrOmax.com']",-1.0,0,NaN,17,generic,0,-1.0,"['g', 'e', 'n', 'e', 'r', 'i', 'c', 'z', 'i', ...",0,3.494082674117968,1 day,6:20:36.431331,com,0.0,0,"['abuse@NameBright.com', 'GenericZithrOmax.com...",0.0,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['gen', 'ene', 'ner', 'eri', 'ric', 'icz', 'cz...","defaultdict(<class 'int'>, {'e': 2, 't': 1, 'o...","['ge', 'en', 'ne', 'er', 'ri', 'ic', 'cz', 'zi...",DropCatch.com 865 LLC,genericzithromax,4.0,0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451465,CN,23848.0,599.0,202.69.26.48,b'www.4008000000.com.',GuangDong,NaN,CN,2007-07-31 10:39:33,0,0,"['4008000000.COM', '4008000000.com']",293612.0,1,PING AN INSURANCE (GROUP) COMPANY OF CHINA LTD,LTD,14,0,4008000000,NaN,"['4', '0', '0', '8', '0', '0', '0', '0', '0', ...",0,1.4814804631846703,4657 days 20:38:15.205623,com,0.0,1,"['registrarcs@huyi.cn', 'pub_dns@pingan.com.cn...",50.0,0,"[('apple.com', 86), ('adobe.com', 86)]",0,"['400', '008', '080', '800', '000', '000', '00...","defaultdict(<class 'int'>, {'0': 8, 'w': 3, '4...","['40', '00', '08', '80', '00', '00', '00', '00...",0,Hu Yi Global Information Hong Kong Limited,3.0,0,CN
472982,RU,44704.0,56.0,193.232.108.67,b'x5.ru.',NaN,NaN,NaN,2005-09-06 20:00:

#### '3gram'

In [78]:
map_col_names = {
'puny_coded' : 'numeric_percentage', 
'typos' : 'puny_coded' ,
'oc_8' : 'typos', 
'3gram' : 'oc_8', 
'char_distribution' : '3gram', 
'2gram' : 'char_distribution', 
'Registrar' : '2gram',
'Name_Server_Count' : 'Registrar',
'numeric_percentage' : 'Name_Server_Count'
}

In [79]:
temp_data_complement = data[~data['3gram'].isin(['0',  0])]
temp_data = data[data['3gram'].isin(['0',  0])]

In [80]:
temp_data.rename(columns=map_col_names, inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1852\2435975881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.rename(columns=map_col_names, inplace=True)


In [81]:
data = pd.concat([temp_data, temp_data_complement], axis = 0)

In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499134 entries, 28423 to 494124
Data columns (total 40 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Country             389247 non-null  object 
 1   ASN                 389304 non-null  object 
 2   TTL                 499134 non-null  float64
 3   IP                  390538 non-null  object 
 4   Domain              499134 non-null  object 
 5   State               220137 non-null  object 
 6   Registrant_Name     13710 non-null   object 
 7   Country.1           256983 non-null  object 
 8   Creation_Date_Time  366046 non-null  object 
 9   hex_32              499011 non-null  object 
 10  hex_8               499134 non-null  object 
 11  Domain_Name         401406 non-null  object 
 12  Alexa_Rank          444716 non-null  object 
 13  subdomain           499122 non-null  object 
 14  Organization        204023 non-null  object 
 15  Page_Rank           443870 non-null

In [83]:
data.describe()

,TTL,dec_8,malware
count,499134.000000,499134.0,499134.000000
mean,4613.544884,0.0,0.010015
std,6687.776794,0.0,0.099574
min,0.000000,0.0,0.000000
25%,299.000000,0.0,0.000000
50%,899.000000,0.0,0.000000
75%,3599.000000,0.0,0.000000
max,397224.000000,0.0,1.000000


## Features engineering

#### 'Country'

In [84]:
data = data.drop(['Registrant_Name', 'Country.1', 'Country'], axis=1)

In [85]:
def country_mode(code):
    if pd.isna(code):
        out = -1
    elif code.isdigit():
        out = -1
    else:
        out = code.upper()
    return out

In [86]:
data['CountryCODE'] = data['CountryCODE'].apply(lambda x: country_mode(x))

In [87]:
data['CountryCODE'].unique()

array(['US', 'CA', -1, 'TR', 'CN', 'EU', 'VG', 'SC', 'FR', 'DK', 'DE',
       'BG', 'AU', 'ES', 'HR', 'NL', 'RO', 'SE', 'IN', 'IR', 'CH', 'CL',
       'HK', 'ID', 'RU', 'IE', 'LU', 'UK', 'GB', 'NO', 'PA', 'KR', 'IT',
       'SG', 'NZ', 'JP', 'BE', 'AT', 'BR', 'CZ', 'KY', 'UA', 'PT', 'PL',
       'ET', 'ZA', 'BY', 'GI', 'TW', 'EE', 'MX', 'FI', 'IS', 'HU', 'CY',
       'PH', 'LT', 'IL', 'AR', 'BS', 'KZ', 'SK', 'CU', 'BZ', 'KH', 'CO',
       'MO', 'MY', 'EG', 'GE', 'MT', 'UG', 'TH', 'PR', 'RS', 'KE', 'SI',
       'PK', 'QA', 'MD', 'JO', 'LV', 'BD', 'JM', 'GR', 'VN', 'GH', 'AZ',
       'AE', 'UY', 'CR', 'MA', 'PE', 'BO', 'NP', 'VE', 'NG', 'EC', 'SA',
       'KN', 'BB', 'BM', 'PY', 'IM', 'CW', 'UN', 'BA', 'MC', 'ML', 'MN',
       'PS', 'LK', 'ZW', 'AM', 'LB', 'VA', 'LI', 'TT', 'GG', 'AD', 'BH',
       'MZ', 'DZ', 'MK', 'RE', 'AF', 'CI', 'UZ', 'PF', 'TZ', 'KG', 'LA',
       'SM', 'OM', 'MU', 'KP', 'LY', 'TJ', 'CV', 'TN', 'JE', 'GY', 'LR',
       'CM', 'NE', 'RW', 'BN', 'BW', 'GP', 'BF', 'PM'

In [88]:
list(data['CountryCODE'].value_counts(normalize=True)[:11].index)

['US', -1, 'DE', 'CN', 'FR', 'RU', 'GB', 'VG', 'CA', 'NL', 'JP']

In [89]:
def country_short(x):
    if x in ['US', -1, 'DE', 'CN', 'FR', 'RU', 'GB', 'VG', 'CA', 'NL', 'JP']:
        return x
    else:
        return -1

data['CountryCODE'] = data['CountryCODE'].apply(country_short)

In [90]:
data = pd.get_dummies(data, columns=['CountryCODE'], drop_first=True, dtype=int )

In [91]:
CountryCODE_features = [feat for feat in data.columns if 'CountryCODE' in feat]

#### 'ASN'

In [92]:
len(data[data['ASN'].isna()])

109830

In [93]:
data['ASN'].value_counts(normalize=True)

ASN
13335.0    0.125031
16509.0    0.056108
14618.0    0.034621
40034.0    0.032758
26496.0    0.027303
             ...   
23888.0    0.000003
56013.0    0.000003
19796.0    0.000003
41853.0    0.000003
11663.0    0.000003
Name: proportion, Length: 10887, dtype: float64

In [94]:
data['ASN'] = data['ASN'].fillna(-1)

#### 'IP'

In [95]:
data['IP'].value_counts(normalize=True)

IP
208.91.197.26     0.007630
208.91.197.25     0.005485
208.91.197.23     0.003336
208.91.197.127    0.003324
208.91.197.24     0.003262
                    ...   
41.208.71.3       0.000003
106.12.24.166     0.000003
148.251.20.56     0.000003
118.97.151.133    0.000003
104.17.70.206     0.000003
Name: proportion, Length: 177867, dtype: float64

In [96]:
len(data[data['IP'].isna()])

108596

In [97]:
data['IP'] = data['IP'].apply(lambda x: -1 if pd.isna(x) or len(str(x).split('.')) == 1 else x)

In [98]:
data['IP_1'] = data['IP'].apply(lambda x: -1 if x == -1 else x.split('.')[0])
data['IP_2'] = data['IP'].apply(lambda x: -1 if x == -1 else x.split('.')[1])
data['IP_3'] = data['IP'].apply(lambda x: -1 if x == -1 else x.split('.')[2])
data['IP_4'] = data['IP'].apply(lambda x: -1 if x == -1 else x.split('.')[3])

In [99]:
data

,ASN,TTL,IP,Domain,State,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4
28423,54290.0,3599.0,142.11.210.249,b'coachoutletcanada.ca.',NaN,2016-12-13 01:59:21,0,0,coachoutletcanada.ca,10356341.0,0,NaN,0.0,18,canada,NaN,-1.0,"['c', 'o', 'a', 'c', 'h', 'o', 'u', 't', 'l', ...",0,3.0074354179086584,1228 days 3:25:00.817737,ca,0.0,0,Please ask the Registrar of Record identified ...,Admin or Other contacts of the queried domain...,0.0,0,"[('canada.ca', 90), ('google.ca', 86)]",0,"['coa', 'oac', 'ach', 'cho', 'hou', 'out', 'ut...","defaultdict(<class 'int'>, {'h': 1, 'u': 1, 'd...",0,"['co', 'oa', 'ac', 'ch', 'ho', 'ou', 'ut', 'tl...",coachoutletcanada,0,0,0,0,0,0,0,0,0,1,0,142,11,210,249
29473,7860.0,3599.0,137.149.3.105,b'upei.ca.',NaN,2000-10-24 17:45:20,0,0,upei.ca,73450.0,0,NaN,0.0,5,ei,NaN,-1.0,"['u', 'p', 'e', 'i']",0,1.8575424759098895,7121 days 12:04:39.256947,ca,0.0,0,Please ask the Registrar of Record identified ...,Admin or Other contacts of the queried domain...,0.0,0,"[('netscape.com', 62), ('oup.com', 57)]",0,"['upe', 'pei']","defaultdict(<class 'int'>, {'e': 1, 'u': 1, 'i...",0,"['up', 'pe', 'ei']",upei,0,1,0,0,0,0,0,0,0,0,0,137,149,3,105
34382,13335.0,299.0,104.18.50.1,b'canadagooseoutletcanada.ca.',NaN,2020-04-10 17:19:44,0,0,canadagooseoutletcanada.ca,-1.0,0,NaN,0.0,24,canada,NaN,-1.0,"['c', 'a', 'n', 'a', 'd', 'a', 'g', 'o', 'o', ...",0,3.1328947920873413,13 days 14:51:54.035851,ca,0.0,0,Please ask the Registrar of Record identified ...,Admin or Other contacts of the queried domain...,0.0,0,"[('canada.ca', 90), ('google.ca', 86)]",0,"['can', 'ana', 'nad', 'ada', 'dag', 'ago', 'go...","defaultdict(<class 'int'>, {'e': 2, 'u': 1, 'o...",0,"['ca', 'an', 'na', 'ad', 'da', 'ag', 'go', 'oo...",canadagooseoutletcanada,0,0,0,0,0,0,0,0,0,1,0,104,18,50,1
34383,-1,299.0,-1,b'canadagooseoutletcanada.ca.',NaN,2020-04-10 17:19:44,0,0,canadagooseoutletcanada.ca,-1.0,0,NaN,0.0,24,canada,NaN,-1.0,"['c', 'a', 'n', 'a', 'd', 'a', 'g', 'o', 'o', ...",0,3.1328947920873413,13 days 14:51:55.167669,ca,0.0,0,Please ask the Registrar of Record identified ...,Admin or Other contacts of the queried domain...,0.0,0,"[('canada.ca', 90), ('google.ca', 86)]",0,"['can', 'ana', 'nad', 'ada', 'dag', 'ago', 'go...","defaultdict(<class 'int'>, {'e': 2, 'u': 1, 'o...",0,"['ca', 'an', 'na', 'ad', 'da', 'ag', 'go', 'oo...",canadagooseoutletcanada,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1
34384,13335.0,299.0,104.18.51.1,b'canadagooseoutletcanada.ca.',NaN,2020-04-10 17:19:44,0,0,canadagooseoutletcanada.ca,-1.0,0,NaN,0.0,24,canada,NaN,-1.0,"['c', 'a', 'n', 'a', 'd', 'a', 'g', 'o', 'o', ...",0,3.1328947920873413,13 days 14:51:56.306234,ca,0.0,0,Please ask the Registrar of Record identified ...,Admin or Other contacts of the queried domain...,0.0,0,"[('canada.ca', 90), ('google.ca', 86)]",0,"['can', 'ana', 'nad', 'ada', 'dag', 'ago', 'go...","defaultdict(<class 'int'>, {'e': 2, 'u': 1, 'o...",0,"['ca', 'an', 'na', 'ad', 'da', 'ag', 'go', 'oo...",canadagooseoutletcanada,0,0,0,0,0,0,0,0,0,1,0,104,18,51,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494024,36351.0,21599.0,174.37.62.109,b'camaro6.com.',Arizona,2007-03-26 05:55:01,0,0,CAMARO6.COM,224596.0,0.0,Domains By Proxy LLC,LLC,8,camaro,camaro6,-1.0,"['c', 'a', 'm', 'a', 'r', 'o', '6']",0.0,2.375,4785 days 18:22:20.057263,com,0.0,0,"['abuse@godaddy.com', 'CAMARO6.COM@domainsbypr...",2.0,7.692308,0,"[('qq.com', 86), ('vk.com', 86)]",0,"['cam', 'am

#### 'tld'

In [100]:
data[data['tld'].isna()]

,ASN,TTL,IP,Domain,State,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,tld,dec_8,dec_32,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4
78212,44112.0,599.0,77.222.62.15,b'admin-moovg.ru.',NaN,2012-04-12 12:07:45,0,0Admin-MOOVG.RU,4301463.0,0Administratsiya munitsipalnogo obrazovaniya O...,12Admin,0,NaN,-1.0,"['a', 'd', 'm', 'i', 'n', '-', 'm', 'o', 'o', ...",NaN,2.9528822923277263,2934 days 14:38:34.256618,ru,0.0,0,NaN,0.0,0,"[('yandex.ru', 86), ('mail.ru', 86)]",NaN,0,"['adm', 'dmi', 'min', 'in-', 'n-m', '-mo', 'mo...","defaultdict(<class 'int'>, {'i': 1, 'o': 2, 'd...","['ad', 'dm', 'mi', 'in', 'n-', '-m', 'mo', 'oo...",RU-CENTER-RUAdmin-moovg,4.0,0,-1.0,NaN,0,0,0,0,0,0,0,0,1,0,0,77,222,62,15
103245,44112.0,599.0,77.222.62.209,b'adminbelka.ru.',NaN,2012-10-31 11:38:02,0,0AdminBELKA.RU,-1.0,0Administratsiya munitsipalnogo obrazovaniya B...,11Admin,0,NaN,-1.0,"['a', 'd', 'm', 'i', 'n', 'b', 'e', 'l', 'k', ...",NaN,2.963119653306635,2733 days 2:10:47.363830,ru,0,0,NaN,0.0,0,"[('mail.ru', 86), ('ok.ru', 86)]",NaN,0,"['adm', 'dmi', 'min', 'inb', 'nbe', 'bel', 'el...","defaultdict(<class 'int'>, {'e': 1, 'i': 1, 'k...","['ad', 'dm', 'mi', 'in', 'nb', 'be', 'el', 'lk...",RU-CENTER-RUAdminbelka,4.0,0,-1.0,NaN,0,0,0,0,0,0,0,0,1,0,0,77,222,62,209
358820,12389.0,3599.0,92.127.158.62,b'adminsuhonoy.ru.',NaN,2014-08-25 01:58:57,0,0AdminSUHONOY.RU,-1.0,0Administratsiya Sukhonoyskogo selsoveta,13Admins,0,NaN,-1.0,"['a', 'd', 'm', 'i', 'n', 's', 'u', 'h', 'o', ...",NaN,3.1080982013610083,2074 days 18:35:48.239013,ru,0,0,NaN,0.0,0,"[('yandex.ru', 86), ('mail.ru', 86)]",NaN,0,"['adm', 'dmi', 'min', 'ins', 'nsu', 'suh', 'uh...","defaultdict(<class 'int'>, {'h': 1, 'u': 1, 'i...","['ad', 'dm', 'mi', 'in', 'ns', 'su', 'uh', 'ho...",RU-CENTER-RUAdminsuhonoy,2.0,0,-1.0,NaN,0,0,0,0,0,0,0,0,1,0,0,92,127,158,62


In [101]:
data['tld'].value_counts(normalize=True)[:11].index.values

array(['com', 'org', 'net', 'ru', 'de', 'co.uk', 'nl', 'fr', 'edu', 'it',
       'pl'], dtype=object)

In [102]:
data['tld'] = data['tld'].apply(lambda x: x if x in ['com', 'org', 'net', 'ru', 'de', 'co.uk', 'nl', 'fr', 'edu', 'it', 'pl'] else -1)

In [103]:
data = pd.get_dummies(data, columns=['tld'], dtype = int, drop_first=True)

#### 'entropy'

In [104]:
data['entropy'].describe()

count               499130
unique                4530
top       2.15413541726763
freq                 13164
Name: entropy, dtype: object

In [105]:
data[data['entropy'].isna()]

,ASN,TTL,IP,Domain,State,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,dec_8,dec_32,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4,tld_co.uk,tld_com,tld_de,tld_edu,tld_fr,tld_it,tld_net,tld_nl,tld_org,tld_pl,tld_ru
3180,60068.0,10065.0,195.181.169.3,b'www.phpmyadmin.net.',NaN,NaN,0,0,NaN,NaN,1,NaN,NaN,14Admin,0,NaN,"['p', 'h', 'p', 'm', 'y', 'a', 'd', 'm', 'i', ...",0,2.910052,NaN,net,0.0,NaN,0.0,NaN,0.0,"[('php.net', 86), ('a8.net', 86)]",0,"['php', 'hpm', 'pmy', 'mya', 'yad', 'adm', 'dm...","defaultdict(<class 'int'>, {'h': 1, 'y': 1, 'i...","['ph', 'hp', 'pm', 'my', 'ya', 'ad', 'dm', 'mi...",-1.0,NaN,phpmyadmin,0,0,0,0,0,1,0,0,0,0,0,195,181,169,3,0,0,0,0,0,0,0,0,0,0,0
297813,46606.0,14399.0,108.179.242.55,b'businessadministrationcertificate.com.',NaN,NaN,0,0,NaN,NaN,0,NaN,NaN,34Administration,0,NaN,"['b', 'u', 's', 'i', 'n', 'e', 's', 's', 'a', ...",0,3.4738775367777603,NaN,com,0.0,NaN,0.0,NaN,0,"[('facebook.com', 86), ('twitter.com', 86)]",0,"['bus', 'usi', 'sin', 'ine', 'nes', 'ess', 'ss...","defaultdict(<class 'int'>, {'e': 3, 't': 4, 'c...","['bu', 'us', 'si', 'in', 'ne', 'es', 'ss', 'sa...",-1.0,NaN,businessadministrationcertificate,0,0,0,0,0,0,0,0,0,1,0,108,179,242,55,0,0,0,0,0,0,0,0,0,0,0
342191,7922.0,2441.0,96.80.247.204,b'intelliadmin.com.',NaN,NaN,0,0,NaN,NaN,0,NaN,NaN,13Admin,0,NaN,"['i', 'n', 't', 'e', 'l', 'l', 'i', 'a', 'd', ...",0,2.742337624271511,NaN,com,0.0,NaN,0.0,NaN,0,"[('google.com', 86), ('apple.com', 86)]",0,"['int', 'nte', 'tel', 'ell', 'lli', 'lia', 'ia...","defaultdict(<class 'int'>, {'t': 1, 'e': 1, 'i...","['in', 'nt', 'te', 'el', 'll', 'li', 'ia', 'ad...",-1.0,NaN,intelliadmin,0,0,0,0,0,0,0,0,0,1,0,96,80,247,204,0,0,0,0,0,0,0,0,0,0,0
462109,134705.0,599.0,116.204.187.12,b'lanrenadmin.com.',NaN,NaN,0,0,NaN,NaN,0,NaN,NaN,12Admin,0,NaN,"['l', 'a', 'n', 'r', 'e', 'n', 'a', 'd', 'm', ...",0,2.723308333814104,NaN,com,0.0,NaN,0.0,NaN,0,"[('google.com', 86), ('apple.com', 86)]",0,"['lan', 'anr', 'nre', 'ren', 'ena', 'nad', 'ad...","defaultdict(<class 'int'>, {'e': 1, 'd': 1, 'i...","['la', 'an', 'nr', 're', 'en', 'na', 'ad', 'dm...",-1.0,NaN,lanrenadmin,0,0,0,0,0,0,0,0,0,0,0,116,204,187,12,0,0,0,0,0,0,0,0,0,0,0


In [106]:
def value_chceck (value):    
    if pd.isna(value):
        return -1
    else:
        try:
            float(value)
        except:
            return -1
        return value        

In [107]:
data['entropy'] = data['entropy'].apply(value_chceck)

In [108]:
data['entropy'] = data['entropy'].astype(float)

#### 'len'

In [109]:
data['len'].value_counts()

len
10                  42626
11                  42125
9                   41179
12                  39055
8                   38618
                    ...  
len                     1
29Administration        1
a                       1
34Administration        1
54                      1
Name: count, Length: 181, dtype: int64

In [110]:
data['len'] = data['len'].apply(value_chceck)

In [111]:
data[data['len'].isna()]

,ASN,TTL,IP,Domain,State,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,dec_8,dec_32,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4,tld_co.uk,tld_com,tld_de,tld_edu,tld_fr,tld_it,tld_net,tld_nl,tld_org,tld_pl,tld_ru


#### 'Alexa_Rank'

In [112]:
data['Alexa_Rank'].value_counts()

Alexa_Rank
-1.0          149575
-1.0            1263
2384.0           236
4.0              164
1.0               78
               ...  
5175422.0          1
685534.0           1
10276739.0         1
368961.0           1
716992.0           1
Name: count, Length: 115292, dtype: int64

In [113]:
len(data[data['Alexa_Rank'].isna()])

54418

In [114]:
data['Alexa_Rank'] = data['Alexa_Rank'].apply(value_chceck)

In [115]:
data['Alexa_Rank'] = data['Alexa_Rank'].astype('float')
data['Alexa_Rank'] = data['Alexa_Rank'].astype('int')

#### 'subdomain'

In [116]:
data['subdomain'].value_counts()

subdomain
0                                        324426
1                                        155219
0.0                                       13362
1.0                                        5627
-1.0                                        134
                                          ...  
177953.0                                      1
1Administracija goroda Nizhnevartovsk         1
5737995.0                                     1
4617048.0                                     1
4222631.0                                     1
Name: count, Length: 181, dtype: int64

In [117]:
data[data['subdomain'].isna()]

,ASN,TTL,IP,Domain,State,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,dec_8,dec_32,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4,tld_co.uk,tld_com,tld_de,tld_edu,tld_fr,tld_it,tld_net,tld_nl,tld_org,tld_pl,tld_ru
25820,34119.0,21599.0,185.27.134.23,b'phpnet.us.',nanAdministrator Administrator,0,0,phpnet.us,250514.0,0,NaN,7,NaN,-1,0,5.0,"['p', 'h', 'p', 'n', 'e', 't']",0,2.1205899331922318,-1.0,us,0.0,NaN,0.0,-1.0,0,"[('php.net', 75), ('rs6.net', 59)]",0,"['php', 'hpn', 'pne', 'net']","defaultdict(<class 'int'>, {'h': 1, 'e': 1, 't...","['ph', 'hp', 'pn', 'ne', 'et']",-1.0,NaN,phpnet,0,0,0,0,0,1,0,0,0,0,0,185,27,134,23,0,0,0,0,0,0,0,0,0,0,0
61969,11787.0,299.0,198.8.33.171,b'pacourts.us.',nanAdministrative Office of PA Courts Administ...,0,0,pacourts.us,126456.0,0,NaN,9,NaN,-1,0,4.0,"['p', 'a', 'c', 'o', 'u', 'r', 't', 's']",0,2.8177111123931664,-1.0,us,0.0,NaN,0.0,-1.0,0,"[('icio.us', 86), ('cryoutcreations.eu', 55)]",0,"['pac', 'aco', 'cou', 'our', 'urt', 'rts']","defaultdict(<class 'int'>, {'t': 1, 'u': 1, 'o...","['pa', 'ac', 'co', 'ou', 'ur', 'rt', 'ts']",-1.0,NaN,pacourts,0,0,0,0,0,0,0,0,0,1,0,198,8,33,171,0,0,0,0,0,0,0,0,0,0,0
95143,55052.0,599.0,207.18.56.58,b'konicaminolta.us.',nanAdministrator Domain,0,0,konicaminolta.us,137153.0,0,NaN,14,NaN,-1,0,2.0,"['k', 'o', 'n', 'i', 'c', 'a', 'm', 'i', 'n', ...",0,2.963972427624918,-1.0,us,0.0,NaN,0.0,-1.0,0,"[('icio.us', 86), ('amzn.to', 51)]",0,"['kon', 'oni', 'nic', 'ica', 'cam', 'ami', 'mi...","defaultdict(<class 'int'>, {'t': 1, 'i': 2, 'k...","['ko', 'on', 'ni', 'ic', 'ca', 'am', 'mi', 'in...",-1.0,NaN,konicaminolta,0,0,0,0,0,0,0,0,0,1,0,207,18,56,58,0,0,0,0,0,0,0,0,0,0,0
146293,48635.0,14399.0,185.182.56.95,b'administratiekantoorklazienaveen.nl.',NaN,NaN,0,0Administratiekantoorklazienaveen.nl,-1.0,0,NaN,33,NaN,-1,0,-1.0,"['a', 'd', 'm', 'i', 'n', 'i', 's', 't', 'r', ...",0,3.486547995238492,0.0,nl,0.0,NaN,0.0,NaN,0,"[('google.nl', 86), ('hostnet.nl', 86)]",0,"['adm', 'dmi', 'min', 'ini', 'nis', 'ist', 'st...","defaultdict(<class 'int'>, {'t': 3, 'e': 4, 'o...","['ad', 'dm', 'mi', 'in', 'ni', 'is', 'st', 'tr...",-1.0,nanAdministratiekantoorklazienaveen,0.0,0,0,0,0,0,0,0,1,0,0,0,185,182,56,95,0,0,0,0,0,0,0,0,0,0,0
177285,34119.0,21599.0,185.27.134.23,b'my-place.us.',nanAdministrator Administrator,0,0,my-place.us,3310030.0,0,NaN,9,NaN,-1,0,5.0,"['m', 'y', '-', 'p', 'l', 'a', 'c', 'e']",0,2.8177111123931664,-1.0,us,0.0,NaN,0.0,-1.0,0,"[('icio.us', 86), ('myspace.com', 64)]",0,"['my-', 'y-p', '-pl', 'pla', 'lac', 'ace']","defaultdict(<class 'int'>, {'e': 1, 'y': 1, '-...","['my', 'y-', '-p', 'pl', 'la', 'ac', 'ce']",-1.0,NaN,my-place,0,0,0,0,0,1,0,0,0,0,0,185,27,134,23,0,0,0,0,0,0,0,0,0,0,0
401789,198610.0,599.0,87.236.16.46,b'adminpower.ru.',NaN,2013-11-14 09:36:14,0,0AdminPOWER.RU,10653692.0,0,NaN,11,NaN,-1,0,-1.0,"['a', 'd', 'm', 'i', 'n', 'p', 'o', 'w', 'e', ...",0,3.1449378351248165,-1.0,ru,0.0,NaN,0.0,NaN,0,"[('mail.ru', 86), ('ok.ru', 86)]",0,"['adm', 'dmi', 'min', 'inp', 'npo', 'pow', 'ow...","defaultdict(<class 'int'>, {'e': 1, 'i': 1, 'o...","['ad', 'dm', 'mi', 'in', 'np', 'po', 'ow', 'we...",-1.0,BEGET-RUAdminpower,6.0,0,0,0,0,0,0,0,0,1,0,0,87,236,16,46,0,0,0,0,0,0,0,0,0,0,0
411493,393676.0,299.0,205.147.88.100,b'amac.us.',nanAdministrative Contact,0,0,"['amac.us', 'AMAC.US']",199554.0,0,NaN,5,NaN,-1,0,8.0,"['a', 'm', 'a', 'c']",0,1.4575424759098898,-1.0,us,0.0,NaN,0.0,-1.0,0,"[('cam.ac.uk', 75), ('amazon.co.uk', 63)]",0,"['ama', 'mac']","defaultdict(<class 'int'>, {'m': 1, 'a': 2, 'c...","['am', 'ma', 'ac']",-1.0,NaN,amac,0,0,0,0,0,0,0,0,0,1,0,205,147,88,100,0,0,0,0,0,0,0,0,0,0,0
411494,393676.0,298.0,205.147.

In [118]:
data['subdomain'] = data['subdomain'].apply(lambda x: x if x in [0, 1, 0.0, 1.0] else -1)

#### 'puny_coded'

In [119]:
data['puny_coded'].value_counts()

puny_coded
0                                                      495755
0.0                                                      2413
1                                                         459
[('miitbeian.gov.cn', 86), ('miibeian.gov.cn', 86)]        54
[('qq.com', 86), ('vk.com', 86)]                           37
                                                        ...  
[('icio.us', 86), ('cryoutcreations.eu', 55)]               1
[('sun.com', 59), ('soundcloud.com', 57)]                   1
[('si.edu', 86), ('umn.edu', 59)]                           1
[('slideshare.net', 86), ('doubleclick.net', 86)]           1
[('icio.us', 86), ('myspace.com', 64)]                      1
Name: count, Length: 96, dtype: int64

In [120]:
data[data['puny_coded'].isna()]

,ASN,TTL,IP,Domain,State,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,dec_8,dec_32,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4,tld_co.uk,tld_com,tld_de,tld_edu,tld_fr,tld_it,tld_net,tld_nl,tld_org,tld_pl,tld_ru


In [121]:
data['puny_coded'] = data['puny_coded'].apply(value_chceck)

In [122]:
data['puny_coded'] = data['puny_coded'].astype(int)

#### 'numeric_percentage'

In [123]:
data['numeric_percentage'].value_counts()

numeric_percentage
0.0                   453456
0.0                    15883
14.285714285714285      1186
16.666666666666664      1122
20.0                    1104
                       ...  
16.216216216216218         1
70.58823529411767          1
22.72727272727273          1
21.538461538461537         1
47.82608695652174          1
Name: count, Length: 274, dtype: int64

In [124]:
data[data['numeric_percentage'].isna()]

,ASN,TTL,IP,Domain,State,Creation_Date_Time,hex_32,hex_8,Domain_Name,Alexa_Rank,subdomain,Organization,Page_Rank,len,longest_word,sld,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,dec_8,dec_32,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,oc_8,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4,tld_co.uk,tld_com,tld_de,tld_edu,tld_fr,tld_it,tld_net,tld_nl,tld_org,tld_pl,tld_ru


In [125]:
data = data.drop(['State' , 'hex_32' , 'hex_8', 'Organization', 'Page_Rank'  , 'longest_word', 'sld', 'dec_8', 'dec_32' , 'oc_8'], axis=1)

#### 'Name_Server_Count'

In [126]:
data['Name_Server_Count'] = data['Name_Server_Count'].apply(value_chceck)
data['Name_Server_Count'] = data['Name_Server_Count'].astype(float)
data['Name_Server_Count'] = data['Name_Server_Count'].apply(lambda x: -1 if x > 1e9 else x)
data['Name_Server_Count'].max()

999777666.0

#### 'Domain_Age'

In [127]:
data['Domain_Age'].value_counts()

Domain_Age
0                            81328
ru                              68
com                             46
org                             20
net                             12
                             ...  
1753 days 10:41:39.041012        1
2894 days 18:12:28.637959        1
2894 days 18:12:27.543769        1
263 days 18:18:34.096849         1
1918 days 2:35:38.040838         1
Name: count, Length: 363228, dtype: int64

In [128]:
len(data[data['Domain_Age'].isna()])

54414

In [129]:
def day_value_chceck (value):    
    if pd.isna(value):
        return -1
    elif 'days' in value:
        return value.split(' days ')[0]
    else:
        return -1

In [130]:
data['Domain_Age'] = data['Domain_Age'].apply(day_value_chceck)
data['Domain_Age'] = data['Domain_Age'].astype(int)

#### 'Domain_Name'

In [131]:
data['Domain_Name'].value_counts()

Domain_Name
gov.uk                 770
ac.uk                  619
HUGEDOMAINS.COM        481
0                      382
FACEBOOK.COM           184
                      ... 
mediakube.fi             1
neu.edu.tr               1
ziva-muzika.cz           1
KAFE-FAZENDA.RU          1
OUTERBANKSVOICE.COM      1
Name: count, Length: 195952, dtype: int64

In [132]:
data[data['Domain'].isna()]

,ASN,TTL,IP,Domain,Creation_Date_Time,Domain_Name,Alexa_Rank,subdomain,len,shortened,1gram,obfuscate_at_sign,entropy,Domain_Age,Emails,Name_Server_Count,numeric_percentage,puny_coded,typos,3gram,char_distribution,oc_32,2gram,Registrar,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4,tld_co.uk,tld_com,tld_de,tld_edu,tld_fr,tld_it,tld_net,tld_nl,tld_org,tld_pl,tld_ru


In [133]:
data['Domain.'] = data['Domain'].apply(lambda x: x.count('.'))

In [134]:
data['Domain.'].max()

16

In [135]:
data = data.drop(columns=['Domain_Name', '1gram', 'obfuscate_at_sign', 'Emails', '3gram', 'char_distribution', 'oc_32', '2gram', 'Registrar'], axis=1)

#### 'Creation_Date_Time'

In [136]:
data.loc[1, 'Creation_Date_Time']

'1997-09-15 04:00:00'

In [137]:
data.loc[1, 'Creation_Date_Time'][5:7]

'09'

In [138]:
data['Creation_Date_Time'] = data['Creation_Date_Time'].replace('before Aug-1996', '1996-01-01 00:00:00')
data['Creation_Date_Time_YM'] = data['Creation_Date_Time'].apply(lambda x: int(str(x)[:4])+int(str(x)[5:7])/12 if len(str(x)) == 19 else -1)

In [139]:
data['Creation_Date_Time_YM'].value_counts()

Creation_Date_Time_YM
-1.000000       133626
 1996.083333      4230
 2019.250000      2782
 2018.833333      2440
 2000.250000      2434
                 ...  
 2020.833333         1
 1986.583333         1
 1987.583333         1
 1985.333333         1
 1985.250000         1
Name: count, Length: 412, dtype: int64

#### 'typos'

In [140]:
data['typos_1'] = data['typos'].apply(lambda x: re.findall(r"[0-9][0-9]", x)[0] if len(re.findall(r"[0-9][0-9]", x))>0 else -1)
data['typos_2'] = data['typos'].apply(lambda x: re.findall(r"[0-9][0-9]", x)[1] if len(re.findall(r"[0-9][0-9]", x))>1 else -1)
data['typos_3'] = data['typos'].apply(lambda x: re.findall(r"[0-9][0-9]", x)[2] if len(re.findall(r"[0-9][0-9]", x))>2 else -1)
data['typos_4'] = data['typos'].apply(lambda x: re.findall(r"[0-9][0-9]", x)[3] if len(re.findall(r"[0-9][0-9]", x))>3 else -1)
data['typos_5'] = data['typos'].apply(lambda x: re.findall(r"[0-9][0-9]", x)[4] if len(re.findall(r"[0-9][0-9]", x))>4 else -1)

In [141]:
data['typos_3'].value_counts()

typos_3
-1    492105
86      6040
51       305
54       122
50        79
45        79
88        42
53        41
60        40
43        33
48        29
44        26
55        21
56        19
52        15
46        14
40        13
90        12
57        12
64        11
16        10
10         8
47         7
49         7
36         7
62         6
68         6
59         5
89         4
73         3
66         2
94         2
69         2
67         1
42         1
63         1
34         1
41         1
39         1
93         1
Name: count, dtype: int64

In [142]:
data['len'] = data['len'].astype(float)
data['numeric_percentage'] = data['numeric_percentage'].astype(float)
data['IP_1'] = data['IP_1'].astype(float)
data['IP_2'] = data['IP_2'].astype(float)
data['IP_3'] = data['IP_3'].astype(float)
data['IP_4'] = data['IP_4'].astype(float)
data['typos_1'] = data['typos_1'].astype(float)
data['typos_2'] = data['typos_2'].astype(float)
data['typos_3'] = data['typos_3'].astype(float)
data['typos_4'] = data['typos_4'].astype(float)
data['typos_5'] = data['typos_5'].astype(float)

In [143]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499134 entries, 28423 to 494124
Data columns (total 48 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ASN                    499134 non-null  object 
 1   TTL                    499134 non-null  float64
 2   IP                     499134 non-null  object 
 3   Domain                 499134 non-null  object 
 4   Creation_Date_Time     366046 non-null  object 
 5   Alexa_Rank             499134 non-null  int32  
 6   subdomain              499134 non-null  float64
 7   len                    499134 non-null  float64
 8   shortened              499112 non-null  object 
 9   entropy                499134 non-null  float64
 10  Domain_Age             499134 non-null  int32  
 11  Name_Server_Count      499134 non-null  float64
 12  numeric_percentage     499134 non-null  float64
 13  puny_coded             499134 non-null  int32  
 14  typos                  499134 non-nul

In [144]:
data_numeric = data.select_dtypes(exclude = object)

In [146]:
data_numeric.describe()

,TTL,Alexa_Rank,subdomain,len,entropy,Domain_Age,Name_Server_Count,numeric_percentage,puny_coded,malware,CountryCODE_CA,CountryCODE_CN,CountryCODE_DE,CountryCODE_FR,CountryCODE_GB,CountryCODE_JP,CountryCODE_NL,CountryCODE_RU,CountryCODE_US,CountryCODE_VG,IP_1,IP_2,IP_3,IP_4,tld_co.uk,tld_com,tld_de,tld_edu,tld_fr,tld_it,tld_net,tld_nl,tld_org,tld_pl,tld_ru,Domain.,Creation_Date_Time_YM,typos_1,typos_2,typos_3,typos_4,typos_5
count,499134.000000,4.991340e+05,499134.000000,499134.000000,499134.000000,499134.000000,4.991340e+05,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000,499134.000000
mean,4613.544884,7.170549e+05,-0.950683,12.096060,2.665938,3444.437472,6.092633e+03,0.864951,-0.000084,0.010015,0.019157,0.047226,0.048113,0.029373,0.025572,0.010306,0.018029,0.027784,0.475530,0.024138,93.181546,90.652757,96.721484,86.335908,0.018330,0.525466,0.025542,0.010835,0.011149,0.010112,0.039160,0.012235,0.079083,0.009100,0.030022,2.415760,1469.650808,82.159296,80.606639,0.159254,-0.986212,-0.999221
std,6687.776794,1.815827e+06,0.263500,4.980625,0.544678,3226.843444,1.692289e+06,4.179104,0.043993,0.099574,0.137077,0.212122,0.214006,0.168850,0.157856,0.100993,0.133057,0.164354,0.499401,0.153477,73.392319,83.959680,84.408959,80.096356,0.134141,0.499352,0.157765,0.103525,0.105000,0.100046,0.193975,0.109934,0.269868,0.094958,0.170648,0.554704,889.241072,12.377416,12.659142,9.810087,0.973878,0.196521
min,0.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-159.000000,-1.000000e+00,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,299.000000,-1.000000e+00,-1.000000,8.000000,2.373267,-1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.000000,13.000000,6.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,-1.000000,86.000000,86.000000,-1.000000,-1.000000,-1.000000
50%,899.000000,3.887250e+04,-1.000000,11.000000,2.742338,2892.000000,2.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,94.000000,71.000000,86.000000,68.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2001.833333,86.000000,86.000000,-1.000000,-1.000000,-1.000000
75%,3599.000000,4.131812e+05,-1.000000,15.000000,3.062772,6445.000000,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,162.000000,168.000000,173.000000,151.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2011.500000,86.000000,86.000000,-1.000000,-1.000000,-1.000000
max,397224.000000,1.091875e+07,1.000000,142.000000,4.766781,12830.000000,9.997777e+08,72.727273,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.000000,255.000000,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,2020.833333,97.000000,97.000000,94.000000,88.000000,57.000000


In [147]:
len(data_numeric[data_numeric['malware']==0])

494135

In [150]:
from sklearn.model_selection  import train_test_split

W trakcie prac przygotowane zostały 3 zestawy danych treningowych oraz testowych:
   1. (X, y) zawierają wszystkie zaimportowane i przygotowane dane 
   2. (X_balanced, y_balanced) zachowują stosunek 40:60 niebezpiecznych domen do bezpiecznych
   3. (X_train_NB, X_test_NB, y_train_NB, y_test_NB) dane treningowe zachowują stosunek 40:60 niebezpiecznych domen do bezpiecznych, natomiast w danych testowych znajdują się wszystkie pozostałe próbki 

In [152]:
X = data_numeric.drop('malware', axis = 1)
y = data_numeric['malware']

In [153]:
len(X), len(y)

(499134, 499134)

In [154]:
bening_balanced_60 = data_numeric[data_numeric['malware']==0].sample(int(len(data_numeric[data_numeric['malware']==1])*60/40))
malware_balanced_40 = data_numeric[data_numeric['malware']==1]

data_numeric_balanced = pd.concat([bening_balanced_60, malware_balanced_40], axis = 0)

X_balanced = data_numeric_balanced.drop('malware', axis = 1)
y_balanced = data_numeric_balanced['malware'] 

In [155]:
len(X_balanced), len(y_balanced)

(12497, 12497)

In [156]:
bening_balanced_all = data_numeric[data_numeric['malware']==0]
malware_balanced_all = data_numeric[data_numeric['malware']==1]


XM = malware_balanced_all.drop('malware', axis = 1)
yM = malware_balanced_all['malware']

XB = bening_balanced_all.drop('malware', axis = 1)
yB = bening_balanced_all['malware']


X_train_M, X_test_M, y_train_M, y_test_M = train_test_split(XM, yM, test_size = 0.2, random_state=0)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(XB, yB, test_size = 0.985, random_state=0)

X_train_NB = pd.concat([X_train_M, X_train_B])
X_test_NB = pd.concat([X_test_M, X_test_B])
y_train_NB = pd.concat([y_train_M, y_train_B])
y_test_NB = pd.concat([y_test_M, y_test_B])

In [157]:
len(X_train_NB), len(X_test_NB), len(y_train_NB), len(y_test_NB)

(11411, 487723, 11411, 487723)

In [158]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

In [159]:
def calculate_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    accB = balanced_accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
   
    return print("\nAccuracy: \t", acc, "\nBalanced Accuracy:\t", accB, '\nf1: \t\t', f1, '\nPrecision: \t', prec, '\nRecall: \t', rec, "\nConfusion Matrix: \n", cm)

## "Logistic Regression"

In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
X_trainB, X_testB, y_trainB, y_testB = train_test_split(X_balanced, y_balanced, test_size = 0.15, random_state=0)

In [161]:
len(X_train), len(X_test), len(y_train), len(y_test)

(349393, 149741, 349393, 149741)

In [162]:
len(X_trainB), len(X_testB), len(y_trainB), len(y_testB)

(10622, 1875, 10622, 1875)

In [163]:
from sklearn.preprocessing import StandardScaler

In [164]:
sc_X = StandardScaler()
sc_XB = StandardScaler()
sc_XNB = StandardScaler()

In [165]:
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

X_trainB = sc_XB.fit_transform(X_trainB)
X_testB = sc_XB.transform(X_testB)

In [166]:
from sklearn.linear_model import LogisticRegression

In [167]:
classifier = LogisticRegression(class_weight='balanced')
classifierB = LogisticRegression(class_weight='balanced')
classifierNB = LogisticRegression(class_weight='balanced')

In [168]:
classifier.fit(X_train, y_train)
classifierB.fit(X_trainB, y_trainB)
classifierNB.fit(X_train_NB, y_train_NB)

y_pred = classifier.predict(X_test)
y_predB = classifierB.predict(X_testB)
y_pred_NB = classifierB.predict(X_test_NB)


C:\Users\Acer\anaconda3\envs\domain-classifier\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Wyniki modelu dla zbioru niezbalansowanego

In [169]:
calculate_metrics(y_test, y_pred)


Accuracy: 	 0.6890297246579093 
Balanced Accuracy:	 0.7128557048863974 
f1: 		 0.04476172892690833 
Precision: 	 0.023081642583620707 
Recall: 	 0.7371621621621621 
Confusion Matrix: 
 [[102085  46176]
 [   389   1091]]


Wyniki modelu dla zbioru zbalansowanego 

In [170]:
calculate_metrics(y_testB, y_predB)


Accuracy: 	 0.696 
Balanced Accuracy:	 0.701700971412581 
f1: 		 0.6643109540636043 
Precision: 	 0.6071044133476857 
Recall: 	 0.7334200260078023 
Confusion Matrix: 
 [[741 365]
 [205 564]]


Wyniki predykcji modelu wytrenowanego na zbalansowanym zbiorze danych na danych niezbalansowanych

In [171]:
calculate_metrics(y_test_NB, y_pred_NB)


Accuracy: 	 0.700463172743545 
Balanced Accuracy:	 0.668796780715109 
f1: 		 0.008645200692158924 
Precision: 	 0.004352133365217095 
Recall: 	 0.637 
Confusion Matrix: 
 [[340995 145728]
 [   363    637]]


## "Random Forest Classification"

In [172]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
X_trainB, X_testB, y_trainB, y_testB = train_test_split(X_balanced, y_balanced, test_size = 0.15, random_state=0)

In [175]:
from sklearn.ensemble import RandomForestClassifier

In [176]:
classifier = RandomForestClassifier(n_estimators = 100, random_state = 0, class_weight='balanced')
classifierB = RandomForestClassifier(n_estimators = 100, random_state = 0, class_weight='balanced')
classifierNB = RandomForestClassifier(n_estimators = 100, random_state = 0, class_weight='balanced')

In [177]:
classifier.fit(X_train, y_train)
classifierB.fit(X_trainB, y_trainB)
classifierNB.fit(X_train_NB, y_train_NB)

y_pred = classifier.predict(X_test)
y_predB = classifierB.predict(X_testB)
y_predNB = classifierNB.predict(X_test_NB)

Wyniki modelu dla zbioru niezbalansowanego

In [178]:
calculate_metrics(y_test, y_pred)


Accuracy: 	 0.9919794845767025 
Balanced Accuracy:	 0.6183382660454345 
f1: 		 0.3688912243825538 
Precision: 	 0.8297872340425532 
Recall: 	 0.23716216216216215 
Confusion Matrix: 
 [[148189     72]
 [  1129    351]]


Wyniki modelu dla zbioru zbalansowanego 

In [179]:
calculate_metrics(y_testB, y_predB)


Accuracy: 	 0.8714666666666666 
Balanced Accuracy:	 0.8710391598492206 
f1: 		 0.8471781864299303 
Precision: 	 0.8267326732673267 
Recall: 	 0.8686605981794538 
Confusion Matrix: 
 [[966 140]
 [101 668]]


Wyniki predykcji modelu wytrenowanego na zbalansowanym zbiorze danych na danych niezbalansowanych

In [180]:
calculate_metrics(y_test_NB, y_predNB)


Accuracy: 	 0.9023298060579469 
Balanced Accuracy:	 0.8412905697902092 
f1: 		 0.031709895113423855 
Precision: 	 0.016183915677649596 
Recall: 	 0.78 
Confusion Matrix: 
 [[439307  47416]
 [   220    780]]


## "K-Nearest Neighbors"

In [181]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
X_trainB, X_testB, y_trainB, y_testB = train_test_split(X_balanced, y_balanced, test_size = 0.15, random_state=0)

In [182]:
sc_X = StandardScaler()
sc_XB = StandardScaler()
sc_XNB = StandardScaler()

In [183]:
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

X_trainB = sc_XB.fit_transform(X_trainB)
X_testB = sc_XB.transform(X_testB)

X_train_NB = sc_XNB.fit_transform(X_train_NB)
X_test_NB = sc_XNB.transform(X_test_NB)

In [184]:
from sklearn.neighbors import KNeighborsClassifier

In [185]:
classifier = KNeighborsClassifier(n_neighbors = 10, weights='distance' )
classifierB = KNeighborsClassifier(n_neighbors = 10, weights='distance')
classifierNB = KNeighborsClassifier(n_neighbors = 10, weights='distance')

classifier.fit(X_train, y_train)
classifierB.fit(X_trainB, y_trainB)
classifierNB.fit(X_train_NB, y_train_NB)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [186]:
y_pred = classifier.predict(X_test)
y_predB = classifierB.predict(X_testB)
y_predNB = classifierNB.predict(X_test_NB)

Wyniki modelu dla zbioru niezbalansowanego

In [187]:
calculate_metrics(y_test, y_pred)


Accuracy: 	 0.9917190348668701 
Balanced Accuracy:	 0.6095106406579923 
f1: 		 0.34391534391534395 
Precision: 	 0.7926829268292683 
Recall: 	 0.2195945945945946 
Confusion Matrix: 
 [[148176     85]
 [  1155    325]]


Wyniki modelu dla zbioru zbalansowanego 

In [188]:
calculate_metrics(y_testB, y_predB)


Accuracy: 	 0.7728 
Balanced Accuracy:	 0.7626400035743092 
f1: 		 0.7182539682539683 
Precision: 	 0.730820995962315 
Recall: 	 0.706111833550065 
Confusion Matrix: 
 [[906 200]
 [226 543]]


Wyniki predykcji modelu wytrenowanego na zbalansowanym zbiorze danych na danych niezbalansowanych

In [189]:
calculate_metrics(y_test_NB, y_predNB)


Accuracy: 	 0.8556762752628029 
Balanced Accuracy:	 0.7650247697355581 
f1: 		 0.01879059912459227 
Precision: 	 0.009528117843309113 
Recall: 	 0.674 
Confusion Matrix: 
 [[416659  70064]
 [   326    674]]


# Wnioski końcowe

Dużym problemem w zbudowaniu działającego modelu było znaczące niezbalansowanie klas. Strategią przyjetą, aby model działał w rzeczywistości było wytrenowanie go na danych zbalansowanych i sprawdznie dokładności dla zbioru niezbalansowanego.

Jako metrykę porównawczą modeli przyjęto F1. Modelem, który najlepiej poradził sobie z zadaniem jest Random Forest Classification. Ewaluacja modelu osiągnęła najwyższy wynik matryki F1 równy 0.845 dla zbalansowanego zbioru danych. 

Podczas predykcji modelu na niezbalansowanym zbiorze danych oczekiwany jest spadek dokładności, wywołanych wzrostem liczby fałszywie pozytywnych predykcji, jednak udało się poprawnie sklasyfikować 78% domen szkodliwych.